# nbextensions recommended:
    

    + Collapsible Headings
    + Hide Input
    + Hide Input All
    + Scratchpad
    + Toggle all line numbers
    + Variable inspector

# <font color=gray>Imports & Data Loading for Simulation Years 255-259</font>

In [ ]:
%matplotlib inline

import bottleneck as bn
import calendar
import cartopy
import cartopy.crs as ccrs
import cmocean.cm as cmo
import glob
import holoviews as hv
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import panel as pn
import xarray as xr

# Disable warning messages
import warnings
warnings.filterwarnings('ignore')

from bisect import bisect_left
from bokeh.palettes import brewer
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
from matplotlib import colors
from palettable.cartocolors.qualitative import Prism_8
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()
pn.extension()

In [ ]:
def closest_lat(sel_lat):
    """
    Returns closest latitude to sel_lat in yt coordinates (assumes dsm.yt_ocean is sorted)
    If two numbers are equally close, return the smallest.
    
    Parameters:
    -----------
        sel_lat: float with the selected latitude
    """
    pos = bisect_left(dsm.yt_ocean, sel_lat)
    if pos == 0:
        return dsm.yt_ocean[0].values
    if pos == len(dsm.yt_ocean):
        return dsm.yt_ocean[-1].values
    before = dsm.yt_ocean[pos - 1].values
    after = dsm.yt_ocean[pos].values
    if after - sel_lat < sel_lat - before:
       return after
    else:
       return before

In [ ]:
def closest_lon(sel_lon):
    """
    Returns closest longitude to sel_lon in xt coordinates (assumes dsm.xt_ocean is sorted)
    If two numbers are equally close, return the smallest.
    
    Parameters:
    -----------
        sel_lon: float with the selected longitude
    """
    pos = bisect_left(dsm.xt_ocean, sel_lon)
    if pos == 0:
        return dsm.xt_ocean[0].values
    if pos == len(dsm.xt_ocean):
        return dsm.xt_ocean[-1].values
    before = dsm.xt_ocean[pos - 1].values
    after = dsm.xt_ocean[pos].values
    if after - sel_lon < sel_lon - before:
       return after
    else:
       return before

In [ ]:
def closest_lon_u(sel_lon):
    """
    Returns closest longitude to sel_lon in xu coordinates (assumes dso.xu_ocean is sorted)
    If two numbers are equally close, return the smallest.
    
    Parameters:
    -----------
        sel_lon: float with the selected longitude
    """
    pos = bisect_left(dso.xu_ocean, sel_lon)
    if pos == 0:
        return dso.xu_ocean[0].values
    if pos == len(dso.xu_ocean):
        return dso.xu_ocean[-1].values
    before = dso.xu_ocean[pos - 1].values
    after = dso.xu_ocean[pos].values
    if after - sel_lon < sel_lon - before:
       return after
    else:
       return before

In [ ]:
def set_inactive_tool(plot, element): 
    """
    For Bokeh plots, deactivates the Hover tool on initial rendering
    """
    plot.state.toolbar.active_inspect = None 

### LOAD DATA FILES

In [ ]:
# Open LOCAL files
dsm = xr.open_dataset('/Users/Alfonso/Documents/payu/output063/wo/ocean_month.nc')
dso = xr.open_dataset('/Users/Alfonso/Documents/payu/output063/wo/ocean.nc')

dsm_c = xr.open_dataset('/Users/Alfonso/Documents/payu/output063/control/ocean_month.nc')
dso_c = xr.open_dataset('/Users/Alfonso/Documents/payu/output063/control/ocean.nc')

#topog = xr.open_dataset('/Users/Alfonso/Documents/payu/topog.nc')
topog = xr.open_dataset('/Users/alfonso/Documents/payu/OM2/topogtools/1deg_maru/topog_ori.nc')
# Rename & copy coordinates from one dataset to another so HoloViews knows which dims/coords are exactly the same
topog = topog.rename({'nx':'xt_ocean', 'ny':'yt_ocean'})
topog.coords['xt_ocean'] = dsm.coords['xt_ocean']
topog.coords['yt_ocean'] = dsm.coords['yt_ocean']

In [ ]:
ds_ice = xr.open_mfdataset('/Users/Alfonso/Documents/payu/output063/wo/cice/*.nc', combine='by_coords')
ds_ice = ds_ice.drop_dims('d2')

ds_ice_c = xr.open_mfdataset('/Users/Alfonso/Documents/payu/output063/control/cice/*.nc', combine='by_coords')

# Make something useful of the useless ni, nj coordinates on the CICE output
ds_ice = ds_ice.rename({'ni':'xt_ocean', 'nj':'yt_ocean'})
ds_ice.coords['xt_ocean'] = dsm.coords['xt_ocean']
ds_ice.coords['yt_ocean'] = dsm.coords['yt_ocean']

ds_ice_c = ds_ice_c.rename({'ni':'xt_ocean', 'nj':'yt_ocean'})
ds_ice_c.coords['xt_ocean'] = dsm.coords['xt_ocean']
ds_ice_c.coords['yt_ocean'] = dsm.coords['yt_ocean']

# Mixed Layer Depth (or Convection Depth)

In [ ]:
# MLD through time, averaged per month
ds_mld = dsm.mld.loc[:,-78:-45,:]
ds_mld = ds_mld.groupby('time.month').mean('time')
# Ctrl MLD through time, averaged per month
ds_mld_c = dsm_c.mld.loc[:,-78:-45,:]
ds_mld_c = ds_mld_c.groupby('time.month').mean('time')

# Bathymetry of the experiment
bathymetry = topog.depth.loc[-78:-45, :]

# Sea-Level through time, averaged per month
ds_sealevel = dsm.sea_level.loc[:,-78:-45,:]
ds_sealevel = ds_sealevel.groupby('time.month').mean('time')
ds_sealevel_c = dsm_c.sea_level.loc[:,-78:-45,:]
ds_sealevel_c = ds_sealevel_c.groupby('time.month').mean('time')

In [ ]:
hv.extension('bokeh')

colorbar_range_mld = (0, ds_mld.max().values.max())
cmap_mld = "cmo.deep_r"
initial_lon = -68.5

# Stream to follow the pointer over the upper plot (and then update the lower plot's cross-section)
posx_mld = hv.streams.PointerX()
# Stream to retrieve the selection box over the upper plot
bounds_mld = hv.streams.BoundsXY()

# ~~~~~ ~~~~~ ~~~~~ ~~~~~ ~~~~~ ^_^ ~~~~~ ~~~~~ ~~~~~ ~~~~~ ~~~~~
# WIDGETS

# Widget to select the MONTH
month_slider_mld = pn.widgets.DiscreteSlider(name='Month', options=ds_mld.month.values.tolist(), 
                                             value=ds_mld.month.values[0], width=500)

# Widget to select the COLORBAR RANGE
colorbar_range_slider_mld = pn.widgets.RangeSlider(name='Colorbar Range', start=0, 
                                                    end=colorbar_range_mld[1], # As weird as it looks, it's WNTBD
                                                    value=colorbar_range_mld, step=100, width=500)

# Widget to activate a DISCRETE COLORBAR (instead of a continuous one)
discrete_colorbar_toggle_mld = pn.widgets.Toggle(name='Show Discrete Colors', width=150)

# Widget to select the NUMBER of BINS of the colorbar
colorbins_slider_mld = pn.widgets.IntSlider(name='Colorbar # Bins', start=5, end=30, step=1, value=15,
                                            width=200)

# Widgets to compute the MLD TIME-SERIES
mld_selection_text = pn.widgets.TextInput(value='Select area to compute time-series...', width=740)

# ~~~~~ ~~~~~ ~~~~~ ~~~~~ ~~~~~ ^_^ ~~~~~ ~~~~~ ~~~~~ ~~~~~ ~~~~~
# WIDGETS's LINKS
        
# Watch for changes on the widgets' values to update the plots
@pn.depends(sel_month=month_slider_mld,
            col_range=colorbar_range_slider_mld,
            discrete_colors=discrete_colorbar_toggle_mld,
            bin_count=colorbins_slider_mld)
def update_plot_mld(sel_month, col_range, discrete_colors, bin_count):
    plot = hv.QuadMesh(ds_mld.sel(month=sel_month), kdims=["xt_ocean", "yt_ocean"], 
                       vdims=hv.Dimension('mld', range=col_range))
    plot.opts(cmap=plt.cm.get_cmap(cmap_mld, bin_count) if discrete_colors else cmap_mld)
    plot.opts(title='Mixed-Layer Depth during ' + calendar.month_name[sel_month], framewise=True, colorbar=True, 
              clabel="m", width=900, height=250, tools=['hover', 'box_select'], ylabel='Latitude (°N)', 
              xlabel='Longitude (°E)', bgcolor='Silver', toolbar="above", active_tools=['box_select'],
              hooks=[set_inactive_tool])
    return plot

# Watch for changes on the widgets' values to update the plots
@pn.depends(sel_month=month_slider_mld)
def update_cross_mld(x, sel_month):
    longitud = closest_lon(x or (initial_lon))

    # BATHYMETRY CROSS-SECTION
    bathy = hv.Area(bathymetry.sel(xt_ocean=closest_lon(x or initial_lon)), vdims="depth", 
                    label="Bathymetry")
    bathy.opts(framewise=True, invert_yaxis=True)

    # MLD CROSS-SECTION
    mixed = hv.Area(ds_mld.sel(xt_ocean=closest_lon(x or initial_lon))[month_slider_mld.value - 1], 
                    vdims="mld", label="Exp.")
    mixed.opts(framewise=True, invert_yaxis=True)

    # CONTROL MLD CROSS-SECTION
    mixed_c = hv.Area(ds_mld_c.sel(xt_ocean=closest_lon(x or initial_lon))[month_slider_mld.value - 1],
                      vdims="mld", label="Ctrl-9091")
    mixed_c.opts(framewise=True, invert_yaxis=True, color="LimeGreen", alpha=0.75)

    mld_lay = bathy * mixed * mixed_c
    mld_lay.opts(title='Mixed-Layer Depth cross-section at %.1f°E'% longitud, ylabel='Depth (m)', 
                 xlabel='Latitude (°N)', width=900, height=450, padding=(0, (0,0.05)), legend_position='bottom_left',
                 toolbar='above')

    return mld_lay

# Watch for changes on the widgets' values to update the plots
@pn.depends(sel_month=month_slider_mld)
def update_sealevel(x, sel_month):
    longitud = closest_lon(x or (initial_lon))

    # SEA-LEVEL CROSS-SECTION
    sl = hv.Area(ds_sealevel.sel(xt_ocean=closest_lon(x or initial_lon))[month_slider_mld.value - 1], 
                    vdims="sea_level", label="Exp.")
    sl.opts(framewise=True, color="Crimson")

    # CONTROL SEA-LEVEL CROSS-SECTION
    sl_c = hv.Area(ds_sealevel_c.sel(xt_ocean=closest_lon(x or initial_lon))[month_slider_mld.value - 1],
                      vdims="sea_level", label="Ctrl-9091")
    sl_c.opts(framewise=True, color="LimeGreen", alpha=0.75)

    sl_lay = sl * sl_c
    sl_lay.opts(title='Sea-Level cross-section at %.1f°E'% longitud, ylabel='Depth (m)', 
                 xlabel='Latitude (°N)', width=900, height=450, padding=(0, (0.05,0)), legend_position='bottom_right',
                 toolbar='above')

    return sl_lay

# Subscriber to update information text and trigger the plotting of the time-series
def update_bounds_mld(bounds):
    if bounds is None:
        mld_selection_text.value = 'Select area to compute time-series'
    else:
        selection = '(' + str(closest_lat(bounds[1])) + ', '+  str(closest_lat(bounds[3])) + \
                    ', ' + str(closest_lon(bounds[0])) + ', '+  str(closest_lon(bounds[2])) + ')'
        mld_selection_text.value = "Experiment's MLD time-series for coordinates: " + selection + " as (lats, lons)"
    
# Watch for changes in the information text to update the time-series
@pn.depends(mld_selection_text)
def update_time_series(texto):
    if texto[0:4] == "Expe":
        bounds = bounds_mld.bounds
        dsm.mld.loc[:, closest_lat(bounds[1]):closest_lat(bounds[3]), closest_lon(bounds[0]):closest_lon(bounds[2])].\
            mean(['yt_ocean', 'xt_ocean']).plot(size=7.7)
        ax = plt.gca()
        ax.set_ylim(ax.get_ylim()[::-1])
    
    curr_fig=plt.gcf()  
    # Remove the margins around the plot
    curr_fig.tight_layout()
    # Prevent pyplot from showing the plot (it would otherwise appear twice on-screen)
    plt.close(curr_fig) 
    return curr_fig
    
# ~~~~~ ~~~~~ ~~~~~ ~~~~~ ~~~~~ ^_^ ~~~~~ ~~~~~ ~~~~~ ~~~~~ ~~~~~
# LONGITUDE LINE
vline_mld = hv.DynamicMap(lambda x: hv.VLine(x or initial_lon), streams=[posx_mld])
vline_mld.opts(color="Crimson", line_dash='dashed')

# U ANTARCTIC PLOT
plot_mld = hv.DynamicMap(update_plot_mld)
# Bind the selection-box Stream to the U Antarctic Plot
bounds_mld.source = plot_mld
# Inform of which function to call when there are changes in the Stream
bounds_mld.add_subscriber(update_bounds_mld)

# U CROSS-SECTION
cross_mld = hv.DynamicMap(update_cross_mld, streams=[posx_mld])

# U CROSS-SECTION
sl_mld = hv.DynamicMap(update_sealevel, streams=[posx_mld])

# ~~~~~ ~~~~~ ~~~~~ ~~~~~ ~~~~~ ^_^ ~~~~~ ~~~~~ ~~~~~ ~~~~~ ~~~~~
# Build the layout, using Spacers as padding amongst elements 
pn.Column(pn.Row(pn.Spacer(width=50), month_slider_mld, discrete_colorbar_toggle_mld), 
          pn.Row(pn.Spacer(width=50), colorbar_range_slider_mld, colorbins_slider_mld),
          pn.Spacer(height=5),
          plot_mld * vline_mld,
          cross_mld,
          sl_mld,
          pn.Spacer(height=5),
          pn.Row(pn.Spacer(width=50), mld_selection_text),
          update_time_series)

# WIND STRESS

In [ ]:
# Bipolar Colormap for the arrows
vector_map = colors.ListedColormap(['RoyalBlue', 'Crimson'])

# Quivers to display the wind
tau_i = dsm.tau_x.loc[:,-78:-45,:].groupby('time.month').mean('time')
tau_j = dsm.tau_y.loc[:,-78:-45,:].groupby('time.month').mean('time')

# Widget to select the MONTH RANGE
month_range_slider_tau = pn.widgets.IntRangeSlider(name='Month Range', start=1, end=12,
                                               value=(1, 12), step=1, width=150)

# Widget to select the SCALE for the vectors
scale_slider_tau = pn.widgets.FloatSlider(name='Vector Scale', start=0.5, end=10, step=0.5, 
                                             value=3, width=150)

# Widget to select the STEP to sample the vectors 
# (otherwise there would be overplotting, i.e. too many on-screen to make sense of it)
step_slider_tau = pn.widgets.IntSlider(name='Sampling Step', start=1, end=6, step=1, 
                                             value=4, width=150)

@pn.depends(sel_scale=scale_slider_tau,
            sel_step=step_slider_tau,
            sel_months=month_range_slider_tau)  
def plot_monthly_currents(sel_scale, sel_step, sel_months):
    plt.figure(figsize=(13.5,4))
    ax = plt.axes(projection=ccrs.Mercator(central_longitude=-80))
    gl = ax.gridlines(crs=ccrs.PlateCarree(), draw_labels=True, linewidth=2, color='gray', alpha=0.3, linestyle='--')
    gl.ylabel_style = {'size': 15, 'color': 'gray'}
    gl.ylabels_right = False
    gl.xlabels_top = False
    gl.xlabels_bottom = False

    ax.add_feature(cartopy.feature.OCEAN, zorder=0, color='Azure')
    ax.add_feature(cartopy.feature.LAND, zorder=0, color='Khaki')
    
    tau_i_m = tau_i.loc[sel_months[0]:sel_months[1]].mean(['month'])[::sel_step, ::sel_step]
    tau_j_m = tau_j.loc[sel_months[0]:sel_months[1]].mean(['month'])[::sel_step, ::sel_step]

    # Base the color of the arrows on the u-velocity (bipolar)
    magnitude = np.sign(tau_i_m)

    taus = ax.quiver(tau_i.xu_ocean[::sel_step].values, tau_i.yu_ocean[::sel_step].values, 
                     tau_i_m.values, tau_j_m.values, 
                     magnitude.values,  scale=sel_scale, cmap=vector_map, 
                     transform=ccrs.PlateCarree())

    plt.title(f"Wind-stress between {calendar.month_name[np.int(sel_months[0])]}–"
              f"{calendar.month_name[np.int(sel_months[1])]}", fontsize=20, pad=20)
    
    curr_fig=plt.gcf()  
    # Since curr_fig.tight_layout() doesn't work for some unholy reason, we're forced to manually 
    # destroy the margins
    curr_fig.subplots_adjust(left=0.03, right=1, top=0.95, bottom=0)
    plt.close(curr_fig) 
    return curr_fig

pn.Column(pn.Row(month_range_slider_tau, scale_slider_tau, step_slider_tau),
          plot_monthly_currents)

### CURL & SHEAR

In [ ]:
tau_i.mean('month').plot(figsize=(20,5))
plt.title("U-WIND STRESS")
plt.show()

In [ ]:
# Calculate the wind-curl values
wind_curl = tau_i.mean('month')

dtau_x_dy = wind_curl.copy()
dtau_x_dy = dtau_x_dy[1:-1] #drop 1st & last rows

# Select the rows "up & down" the latitude of interest so numpy can broadcast the operation
wind_up = wind_curl[2:]
wind_down = wind_curl[0:-2]

# (dTx(y+1) - dTx(y-1)) / dy
dtau_x_dy.values = (wind_up.values - wind_down.values) \
                   / ((wind_up.yu_ocean.values - wind_down.yu_ocean.values).reshape(59,1) * 111000)   

# The cheap way (the easiest to understand), looping of course...
# for j, lat_taus in enumerate(wind_curl):
#     if ((j > 0) & (j < 59)):
#         dtau_x_dy[j] = (wind_curl[j+1] - wind_curl[j-1]) / (wind_curl.yu_ocean[j+1] - wind_curl.yu_ocean[j-1]) * 111000   

dtau_x_dy.plot(figsize=(20,5))
plt.title("U-WIND SHEAR (dTx/dy)")
plt.show()

# CURRENTS' Vectors

In [ ]:
# Monthly mean currents, through depth
ds_u_monthly = dsm.u.loc[:,:,-78:-45,:].groupby('time.month').mean('time')
ds_v_monthly = dsm.v.loc[:,:,-78:-45,:].groupby('time.month').mean('time')

ds_u_monthly_c = dsm_c.u.loc[:,:,-78:-45,:].groupby('time.month').mean('time')
ds_v_monthly_c = dsm_c.v.loc[:,:,-78:-45,:].groupby('time.month').mean('time')

ds_u_monthly_dif = ds_u_monthly - ds_u_monthly_c
ds_v_monthly_dif = ds_v_monthly - ds_v_monthly_c

In [ ]:
# Replace NaNs with 0s when doing the substraction
ds_u_monthly_dif = ds_u_monthly - ds_u_monthly_c.fillna(0)
ds_v_monthly_dif = ds_v_monthly - ds_v_monthly_c.fillna(0)

In [ ]:
# Bipolar Colormap for the arrows
vector_map = colors.ListedColormap(['RoyalBlue', 'Crimson'])

# Widget to select the DEPTH RANGE
depth_range_slider_u = pn.widgets.IntRangeSlider(name='Depth Range', start=0,
                                                      end=np.int(np.rint(ds_u_monthly.st_ocean.values[-1])),
                                                      value=(0, 100), step=10, width=450)

# Widget to select the MONTH RANGE
month_range_slider_u = pn.widgets.IntRangeSlider(name='Month Range', start=1, end=12,
                                               value=(7, 11), step=1, width=150)

# Widget to select the SCALE for the vectors
scale_slider_u = pn.widgets.FloatSlider(name='Vector Scale', start=0.5, end=10, step=0.5, 
                                             value=2.5, width=150)

# Widget to select the STEP to sample the vectors 
# (otherwise there would be overplotting, i.e. too many on-screen to make sense of it)
step_slider_u = pn.widgets.IntSlider(name='Sampling Step', start=1, end=6, step=1, 
                                             value=3, width=150)

@pn.depends(sel_depths=depth_range_slider_u, 
            sel_scale=scale_slider_u,
            sel_step=step_slider_u,
            sel_months=month_range_slider_u)  
def plot_monthly_currents(sel_depths, sel_scale, sel_step, sel_months):
    plt.figure(figsize=(13.5,12))
    
    
    # ~~~~~ ~~~~~ ~~~~~ ~~~~~ ~~~~~ ^_^ ~~~~~ ~~~~~ ~~~~~ ~~~~~ ~~~~~
    # CTRL currents
    ax_ctrl = plt.subplot(3, 1, 1, projection=ccrs.Mercator(central_longitude=-80))
    
    ax_ctrl.add_feature(cartopy.feature.OCEAN, zorder=0, color='Azure')
    ax_ctrl.add_feature(cartopy.feature.LAND, zorder=0, color='Khaki')

    # Compute mean velocities
    u_vels_ctrl = ds_u_monthly_c.loc[sel_months[0]:sel_months[1], sel_depths[0]:sel_depths[1]].\
                                 mean(['month','st_ocean'])[::sel_step, ::sel_step]
    v_vels_ctrl = ds_v_monthly_c.loc[sel_months[0]:sel_months[1], sel_depths[0]:sel_depths[1]].\
                                 mean(['month','st_ocean'])[::sel_step, ::sel_step]
    # Base the color of the arrows on the direction of the u-velocity (bipolar)
    flow_directions_ctrl = np.sign(u_vels_ctrl)
    ax_ctrl.quiver(ds_u_monthly_c.xu_ocean[::sel_step].values, ds_u_monthly_c.yu_ocean[::sel_step].values,
                   u_vels_ctrl.values, v_vels_ctrl.values, flow_directions_ctrl, scale=sel_scale, 
                   cmap=vector_map, transform=ccrs.PlateCarree())
    
    ax_ctrl.set_title(f"CTRL-9091 Currents from {sel_depths[0]:d} to {sel_depths[1]:d} (m) depth, "
                      f"between {calendar.month_name[np.int(sel_months[0])]}–"
                      f"{calendar.month_name[np.int(sel_months[1])]}", 
                      fontsize=20, pad=20)
    
    
    # ~~~~~ ~~~~~ ~~~~~ ~~~~~ ~~~~~ ^_^ ~~~~~ ~~~~~ ~~~~~ ~~~~~ ~~~~~
    # Experiment's currents
    ax_exp = plt.subplot(3, 1, 2, projection=ccrs.Mercator(central_longitude=-80))
    
    ax_exp.add_feature(cartopy.feature.OCEAN, zorder=0, color='Azure')
    ax_exp.add_feature(cartopy.feature.LAND, zorder=0, color='Khaki')

    # Compute mean velocities
    u_vels_exp = ds_u_monthly.loc[sel_months[0]:sel_months[1], sel_depths[0]:sel_depths[1]].\
                               mean(['month','st_ocean'])[::sel_step, ::sel_step]
    v_vels_exp = ds_v_monthly.loc[sel_months[0]:sel_months[1], sel_depths[0]:sel_depths[1]].\
                               mean(['month','st_ocean'])[::sel_step, ::sel_step]
    
    # Base the color of the arrows on the direction of the u-velocity (bipolar)
    flow_directions_exp = np.sign(u_vels_exp)
    ax_exp.quiver(ds_u_monthly.xu_ocean[::sel_step].values, ds_u_monthly.yu_ocean[::sel_step].values,
                   u_vels_exp.values, v_vels_exp.values, flow_directions_exp, scale=sel_scale, 
                   cmap=vector_map, transform=ccrs.PlateCarree())
    
    ax_exp.set_title(f"EXP.'s Currents from {sel_depths[0]:d} to {sel_depths[1]:d} (m) depth, "
                      f"between {calendar.month_name[np.int(sel_months[0])]}–"
                      f"{calendar.month_name[np.int(sel_months[1])]}", 
                      fontsize=20, pad=20)
    
    
    # ~~~~~ ~~~~~ ~~~~~ ~~~~~ ~~~~~ ^_^ ~~~~~ ~~~~~ ~~~~~ ~~~~~ ~~~~~
    # DiFFERENCES
    ax_dif = plt.subplot(3, 1, 3, projection=ccrs.Mercator(central_longitude=-80))
    
    ax_dif.add_feature(cartopy.feature.OCEAN, zorder=0, color='Azure')
    ax_dif.add_feature(cartopy.feature.LAND, zorder=0, color='Khaki')

    # Compute mean velocities
    u_vels_dif = ds_u_monthly_dif.loc[sel_months[0]:sel_months[1], sel_depths[0]:sel_depths[1]].\
                                  mean(['month','st_ocean'])[::sel_step, ::sel_step]
    v_vels_dif = ds_v_monthly_dif.loc[sel_months[0]:sel_months[1], sel_depths[0]:sel_depths[1]].\
                                  mean(['month','st_ocean'])[::sel_step, ::sel_step]
    # Base the color of the arrows on the direction of the u-velocity (bipolar)
    flow_directions_dif = np.sign(u_vels_dif)
    ax_dif.quiver(ds_u_monthly_dif.xu_ocean[::sel_step].values, ds_u_monthly_dif.yu_ocean[::sel_step].values,
                  u_vels_dif.values, v_vels_dif.values, flow_directions_dif, scale=sel_scale, 
                  cmap=vector_map, transform=ccrs.PlateCarree())
    
    ax_dif.set_title(f"DIFFERENCES in Currents from {sel_depths[0]:d} to {sel_depths[1]:d} (m) depth, "
                     f"between {calendar.month_name[np.int(sel_months[0])]}–"
                     f"{calendar.month_name[np.int(sel_months[1])]}", 
                     fontsize=20, pad=20)
    
    # ~~~~~ ~~~~~ ~~~~~ ~~~~~ ~~~~~ ^_^ ~~~~~ ~~~~~ ~~~~~ ~~~~~ ~~~~~
    curr_fig=plt.gcf()  
    # Since curr_fig.tight_layout() doesn't work for some unholy reason, we're forced to manually 
    # destroy the margins
    curr_fig.subplots_adjust(left=0, right=1, top=0.95, bottom=0)
    plt.close(curr_fig) 
    return curr_fig

pn.Column(pn.Row(depth_range_slider_u, month_range_slider_u, scale_slider_u, step_slider_u),
          plot_monthly_currents)

# U-VELOCITY Averaged within a Depth-Range & Month-Range

In [ ]:
# U-velocity through time, averaged per month
ds_acc = dsm.u.loc[:,:,-78:-45,:]
ds_acc = ds_acc.groupby('time.month').mean('time')
# Ctrl U-velocity averaged through time
ds_acc_c = dsm_c.u.loc[:,:,-78:-45,:]
ds_acc_c = ds_acc_c.groupby('time.month').mean('time')

# Differences
ds_acc_dif = ds_acc - ds_acc_c.fillna(0)

In [ ]:
hv.extension('bokeh')

colorbar_range_acc_avg = (-0.25, 0.25)
colorbar_range_acc_dif = (-0.05, 0.05)
cmap_acc_avg = "cmo.curl"
cmap_acc_dif = "RdBu_r"
initial_lon = -68.5

# Stream to follow the pointer over the upper plot (and then update the lower plot's cross-section)
posx_acc_avg = hv.streams.PointerX()

# ~~~~~ ~~~~~ ~~~~~ ~~~~~ ~~~~~ ^_^ ~~~~~ ~~~~~ ~~~~~ ~~~~~ ~~~~~
# WIDGETS

# Widget to select the MONTH
month_slider_acc_avg = pn.widgets.IntRangeSlider(name='Month Range', start=1, end=12,
                                                 value=(1, 12), step=1, width=250)

# Widget to select the DEPTH
depth_range_acc_avg = pn.widgets.IntRangeSlider(name='Depth Range', start=0,
                                             end=np.int(np.rint(ds_acc.st_ocean.values[-1])),
                                             value=(0, 100), step=10, width=500)

# Widget to select the COLORBAR RANGE
colorbar_slider_acc_avg = pn.widgets.FloatSlider(name='Colorbar Range', start=0, end=1, step=0.05, 
                                             value=colorbar_range_acc_avg[1], width=370)

# Widget to activate a DISCRETE COLORBAR (instead of a continuous one)
discrete_colorbar_toggle_acc_avg = pn.widgets.Toggle(name='Discrete Colors', width=100)

# Widget to select the NUMBER of BINS of the colorbar
colorbins_slider_acc_avg = pn.widgets.IntSlider(name='Colorbar # Bins', start=5, end=30, step=1, value=15,
                                            width=250)

# Widget to select the COLORBAR RANGE
colorbar_slider_acc_dif = pn.widgets.FloatSlider(name='Colorbar Range', start=0, end=0.1, step=0.01, 
                                                 value=colorbar_range_acc_dif[1], width=370)

# ~~~~~ ~~~~~ ~~~~~ ~~~~~ ~~~~~ ^_^ ~~~~~ ~~~~~ ~~~~~ ~~~~~ ~~~~~
# WIDGETS's LINKS
        
# Attach a link to the widget value, so it automatically refreshes the horizontal, 
# depth line upon change
@pn.depends(depth=depth_range_acc_avg)
def update_hline_acc_avg_upper(depth):
    return hv.HLine(depth[0])

@pn.depends(depth=depth_range_acc_avg)
def update_hline_acc_avg_lower(depth):
    return hv.HLine(depth[1])


# EXPERIMENT
# Watch for changes on the widgets' values to update the plots
@pn.depends(sel_month=month_slider_acc_avg,
            depth=depth_range_acc_avg, 
            col_range=colorbar_slider_acc_avg,
            discrete_colors=discrete_colorbar_toggle_acc_avg,
            bin_count=colorbins_slider_acc_avg)
def update_plot_acc_avg(sel_month, depth, col_range, discrete_colors, bin_count):
    plot = hv.QuadMesh(ds_acc.loc[sel_month[0]:sel_month[1], depth[0]:depth[1]].mean(['month','st_ocean']), 
                       kdims=["xu_ocean", "yu_ocean"], vdims=hv.Dimension('u', range=(-col_range, col_range)))
    plot.opts(cmap=plt.cm.get_cmap(cmap_acc_avg, bin_count) if discrete_colors else cmap_acc_avg)
    plot.opts(title=f'EXP. U-velocity between {depth[0]:d}–{depth[1]:d} (m) depth, '
              f'{calendar.month_name[np.int(sel_month[0])]}–{calendar.month_name[np.int(sel_month[1])]}', 
              framewise=True, colorbar=True, 
              clabel="m/s", width=900, height=250, tools=['hover'], ylabel='Latitude (°N)', xlabel='Longitude (°E)', 
              bgcolor='Silver', toolbar="above", hooks=[set_inactive_tool], active_tools=['box_zoom'])
        
    return plot

# Watch for changes on the widgets' values to update the plots
@pn.depends(sel_month=month_slider_acc_avg,
            col_range=colorbar_slider_acc_avg,
            discrete_colors=discrete_colorbar_toggle_acc_avg,
            bin_count=colorbins_slider_acc_avg)
def update_cross_acc_avg(x, sel_month, col_range, discrete_colors, bin_count):
    longitud = closest_lon_u(x or (initial_lon))
    
    plot = hv.QuadMesh(ds_acc.sel(month=slice(sel_month[0],sel_month[1]), xu_ocean=longitud).mean('month'), 
                       kdims=["yu_ocean", "st_ocean"], 
                       vdims=hv.Dimension('u', range=(-col_range, col_range)))
    plot.opts(cmap=plt.cm.get_cmap(cmap_acc_avg, bin_count) if discrete_colors else cmap_acc_avg)
    plot.opts(title='EXP. U-velocity cross-section at %d°E'% (longitud), framewise=True, invert_yaxis=True, 
              colorbar=True, clabel="m/s", bgcolor='Silver', tools=['hover'], width=900, height=450,
              ylabel='Depth (m)', xlabel='Latitude (°N)', toolbar="above", hooks=[set_inactive_tool],
              active_tools=['box_zoom'])
    return plot


# CTRL-9091
# Watch for changes on the widgets' values to update the plots
@pn.depends(sel_month=month_slider_acc_avg,
            depth=depth_range_acc_avg, 
            col_range=colorbar_slider_acc_avg,
            discrete_colors=discrete_colorbar_toggle_acc_avg,
            bin_count=colorbins_slider_acc_avg)
def update_plot_acc_c(sel_month, depth, col_range, discrete_colors, bin_count):
    plot = hv.QuadMesh(ds_acc_c.loc[sel_month[0]:sel_month[1], depth[0]:depth[1]].mean(['month','st_ocean']), 
                       kdims=["xu_ocean", "yu_ocean"], vdims=hv.Dimension('u', range=(-col_range, col_range)))
    plot.opts(cmap=plt.cm.get_cmap(cmap_acc_avg, bin_count) if discrete_colors else cmap_acc_avg)
    plot.opts(title=f'CTRL-9091 U-velocity between {depth[0]:d}–{depth[1]:d} (m) depth, '
              f'{calendar.month_name[np.int(sel_month[0])]}–{calendar.month_name[np.int(sel_month[1])]}', 
              framewise=True, colorbar=True, 
              clabel="m/s", width=900, height=250, tools=['hover'], ylabel='Latitude (°N)', xlabel='Longitude (°E)', 
              bgcolor='Silver', toolbar="above", hooks=[set_inactive_tool], active_tools=['box_zoom'])
        
    return plot

# Watch for changes on the widgets' values to update the plots
@pn.depends(sel_month=month_slider_acc_avg,
            col_range=colorbar_slider_acc_dif,
            discrete_colors=discrete_colorbar_toggle_acc_avg,
            bin_count=colorbins_slider_acc_avg)
def update_cross_acc_c(x, sel_month, col_range, discrete_colors, bin_count):
    longitud = closest_lon_u(x or (initial_lon))
    
    plot = hv.QuadMesh(ds_acc_c.sel(month=slice(sel_month[0],sel_month[1]), xu_ocean=longitud).mean('month'), 
                       kdims=["yu_ocean", "st_ocean"], 
                       vdims=hv.Dimension('u', range=(-col_range, col_range)))
    plot.opts(cmap=plt.cm.get_cmap(cmap_acc_avg, bin_count) if discrete_colors else cmap_acc_avg)
    plot.opts(title='CTRL-9091 U-velocity cross-section at %d°E'% (longitud), framewise=True, invert_yaxis=True, 
              colorbar=True, clabel="m/s", bgcolor='Silver', tools=['hover'], width=900, height=450,
              ylabel='Depth (m)', xlabel='Latitude (°N)', toolbar="above", hooks=[set_inactive_tool],
              active_tools=['box_zoom'])
    return plot


# DIFFERENCES
# Watch for changes on the widgets' values to update the plots
@pn.depends(sel_month=month_slider_acc_avg,
            depth=depth_range_acc_avg,  
            col_range=colorbar_slider_acc_dif,
            discrete_colors=discrete_colorbar_toggle_acc_avg,
            bin_count=colorbins_slider_acc_avg)
def update_plot_acc_dif(sel_month, depth, col_range, discrete_colors, bin_count):
    plot = hv.QuadMesh(ds_acc_dif.loc[sel_month[0]:sel_month[1], depth[0]:depth[1]].mean(['month','st_ocean']), 
                       kdims=["xu_ocean", "yu_ocean"], vdims=hv.Dimension('u', range=(-col_range, col_range)))
    plot.opts(cmap=plt.cm.get_cmap(cmap_acc_dif, bin_count) if discrete_colors else cmap_acc_dif)
    plot.opts(title=f'DiFFERENCES in U-velocity between {depth[0]:d}–{depth[1]:d} (m) depth, '
              f'{calendar.month_name[np.int(sel_month[0])]}–{calendar.month_name[np.int(sel_month[1])]}', 
              framewise=True, colorbar=True, 
              clabel="m/s", width=900, height=250, tools=['hover'], ylabel='Latitude (°N)', xlabel='Longitude (°E)', 
              bgcolor='Silver', toolbar="above", hooks=[set_inactive_tool], active_tools=['box_zoom'])
        
    return plot

# Watch for changes on the widgets' values to update the plots
@pn.depends(sel_month=month_slider_acc_avg, 
            col_range=colorbar_slider_acc_dif,
            discrete_colors=discrete_colorbar_toggle_acc_avg,
            bin_count=colorbins_slider_acc_avg)
def update_cross_acc_dif(x, sel_month, col_range, discrete_colors, bin_count):
    longitud = closest_lon_u(x or (initial_lon))
    
    plot = hv.QuadMesh(ds_acc_dif.sel(month=slice(sel_month[0],sel_month[1]), xu_ocean=longitud).mean('month'), 
                       kdims=["yu_ocean", "st_ocean"], 
                       vdims=hv.Dimension('u', range=(-col_range, col_range)))
    plot.opts(cmap=plt.cm.get_cmap(cmap_acc_dif, bin_count) if discrete_colors else cmap_acc_dif)
    plot.opts(title='DiFFERENCES in U-velocity cross-section at %d°E'% (longitud), framewise=True, invert_yaxis=True, 
              colorbar=True, clabel="m/s", bgcolor='Silver', tools=['hover'], width=900, height=450,
              ylabel='Depth (m)', xlabel='Latitude (°N)', toolbar="above", hooks=[set_inactive_tool],
              active_tools=['box_zoom'])
    return plot

# ~~~~~ ~~~~~ ~~~~~ ~~~~~ ~~~~~ ^_^ ~~~~~ ~~~~~ ~~~~~ ~~~~~ ~~~~~
# LONGITUDE LINE

vline_acc_avg = hv.DynamicMap(lambda x: hv.VLine(x or initial_lon), streams=[posx_acc_avg])
vline_acc_avg.opts(color="RoyalBlue", line_dash='dashed')

# DEPTH LINES
hline_acc_avg_upper = hv.DynamicMap(update_hline_acc_avg_upper)
hline_acc_avg_upper.opts(color="RoyalBlue", line_dash='dashed')

hline_acc_avg_lower = hv.DynamicMap(update_hline_acc_avg_lower)
hline_acc_avg_lower.opts(color="RoyalBlue", line_dash='dashed')

# U ANTARCTIC PLOT
plot_acc_avg = hv.DynamicMap(update_plot_acc_avg)
# U CROSS-SECTION
cross_acc_avg = hv.DynamicMap(update_cross_acc_avg, streams=[posx_acc_avg])

# CTRL U ANTARCTIC PLOT
plot_acc_c = hv.DynamicMap(update_plot_acc_c)
# CTRL U CROSS-SECTION
cross_acc_c = hv.DynamicMap(update_cross_acc_c, streams=[posx_acc_avg])

# DIFFERENCES U ANTARCTIC PLOT
plot_acc_dif = hv.DynamicMap(update_plot_acc_dif)
# DIFFERENCES U CROSS-SECTION
cross_acc_dif = hv.DynamicMap(update_cross_acc_dif, streams=[posx_acc_avg])

# ~~~~~ ~~~~~ ~~~~~ ~~~~~ ~~~~~ ^_^ ~~~~~ ~~~~~ ~~~~~ ~~~~~ ~~~~~
# Build the layout
pn.Column(pn.Row(pn.Spacer(width=50), depth_range_acc_avg, month_slider_acc_avg), 
          pn.Row(pn.Spacer(width=50), colorbar_slider_acc_avg, discrete_colorbar_toggle_acc_avg, 
                 pn.Spacer(width=2), colorbins_slider_acc_avg),
          pn.Spacer(height=5),
          plot_acc_avg * vline_acc_avg,
          cross_acc_avg * hline_acc_avg_upper * hline_acc_avg_lower,
          plot_acc_c * vline_acc_avg,
          cross_acc_c * hline_acc_avg_upper * hline_acc_avg_lower,
          plot_acc_dif * vline_acc_avg,
          cross_acc_dif * hline_acc_avg_upper * hline_acc_avg_lower,
          pn.Row(pn.Spacer(width=50), colorbar_slider_acc_dif))

# V-VELOCITY Averaged within a Depth-Range & Month-Range

In [ ]:
# U-velocity through time, averaged per month
ds_v = dsm.v.loc[:,:,-78:-45,:]
ds_v = ds_v.groupby('time.month').mean('time')
# Ctrl U-velocity averaged through time
ds_v_c = dsm_c.v.loc[:,:,-78:-45,:]
ds_v_c = ds_v_c.groupby('time.month').mean('time')

# Differences
ds_v_dif = ds_v - ds_v_c

In [ ]:
hv.extension('bokeh')

colorbar_range_v_avg = (-0.25, 0.25)
colorbar_range_v_dif = (-0.05, 0.05)
cmap_v_avg = "cmo.curl"
cmap_v_dif = "RdBu_r"
initial_lon = -68.5

# Stream to follow the pointer over the upper plot (and then update the lower plot's cross-section)
posx_v_avg = hv.streams.PointerX()

# ~~~~~ ~~~~~ ~~~~~ ~~~~~ ~~~~~ ^_^ ~~~~~ ~~~~~ ~~~~~ ~~~~~ ~~~~~
# WIDGETS

# Widget to select the MONTH
month_slider_v_avg = pn.widgets.IntRangeSlider(name='Month Range', start=1, end=12,
                                                 value=(1, 12), step=1, width=250)

# Widget to select the DEPTH
depth_range_v_avg = pn.widgets.IntRangeSlider(name='Depth Range', start=0,
                                             end=np.int(np.rint(ds_v.st_ocean.values[-1])),
                                             value=(0, 200), step=10, width=500)

# Widget to select the COLORBAR RANGE
colorbar_slider_v_avg = pn.widgets.FloatSlider(name='Colorbar Range', start=0, end=1, step=0.05, 
                                             value=colorbar_range_v_avg[1], width=370)

# Widget to activate a DISCRETE COLORBAR (instead of a continuous one)
discrete_colorbar_toggle_v_avg = pn.widgets.Toggle(name='Discrete Colors', width=100)

# Widget to select the NUMBER of BINS of the colorbar
colorbins_slider_v_avg = pn.widgets.IntSlider(name='Colorbar # Bins', start=5, end=30, step=1, value=15,
                                            width=250)

# Widget to select the COLORBAR RANGE
colorbar_slider_v_dif = pn.widgets.FloatSlider(name='Colorbar Range', start=0, end=0.1, step=0.01, 
                                                 value=colorbar_range_v_dif[1], width=370)

# ~~~~~ ~~~~~ ~~~~~ ~~~~~ ~~~~~ ^_^ ~~~~~ ~~~~~ ~~~~~ ~~~~~ ~~~~~
# WIDGETS's LINKS
        
# Attach a link to the widget value, so it automatically refreshes the horizontal, 
# depth line upon change
@pn.depends(depth=depth_range_v_avg)
def update_hline_v_avg_upper(depth):
    return hv.HLine(depth[0])

@pn.depends(depth=depth_range_v_avg)
def update_hline_v_avg_lower(depth):
    return hv.HLine(depth[1])


# EXPERIMENT
# Watch for changes on the widgets' values to update the plots
@pn.depends(sel_month=month_slider_v_avg,
            depth=depth_range_v_avg, 
            col_range=colorbar_slider_v_avg,
            discrete_colors=discrete_colorbar_toggle_v_avg,
            bin_count=colorbins_slider_v_avg)
def update_plot_v_avg(sel_month, depth, col_range, discrete_colors, bin_count):
    plot = hv.QuadMesh(ds_v.loc[sel_month[0]:sel_month[1], depth[0]:depth[1]].mean(['month','st_ocean']), 
                       kdims=["xu_ocean", "yu_ocean"], vdims=hv.Dimension('u', range=(-col_range, col_range)))
    plot.opts(cmap=plt.cm.get_cmap(cmap_v_avg, bin_count) if discrete_colors else cmap_v_avg)
    plot.opts(title=f'EXP. V-velocity between {depth[0]:d}–{depth[1]:d} (m) depth, '
              f'{calendar.month_name[np.int(sel_month[0])]}–{calendar.month_name[np.int(sel_month[1])]}', 
              framewise=True, colorbar=True, 
              clabel="m/s", width=900, height=250, tools=['hover'], ylabel='Latitude (°N)', xlabel='Longitude (°E)', 
              bgcolor='Silver', toolbar="above", hooks=[set_inactive_tool], active_tools=['box_zoom'])
        
    return plot

# Watch for changes on the widgets' values to update the plots
@pn.depends(sel_month=month_slider_v_avg,
            col_range=colorbar_slider_v_avg,
            discrete_colors=discrete_colorbar_toggle_v_avg,
            bin_count=colorbins_slider_v_avg)
def update_cross_v_avg(x, sel_month, col_range, discrete_colors, bin_count):
    longitud = closest_lon_u(x or (initial_lon))
    
    plot = hv.QuadMesh(ds_v.sel(month=slice(sel_month[0],sel_month[1]), xu_ocean=longitud).mean('month'), 
                       kdims=["yu_ocean", "st_ocean"], 
                       vdims=hv.Dimension('u', range=(-col_range, col_range)))
    plot.opts(cmap=plt.cm.get_cmap(cmap_v_avg, bin_count) if discrete_colors else cmap_v_avg)
    plot.opts(title='EXP. V-velocity cross-section at %d°E'% (longitud), framewise=True, invert_yaxis=True, 
              colorbar=True, clabel="m/s", bgcolor='Silver', tools=['hover'], width=900, height=450,
              ylabel='Depth (m)', xlabel='Latitude (°N)', toolbar="above", hooks=[set_inactive_tool],
              active_tools=['box_zoom'])
    return plot


# CTRL-9091
# Watch for changes on the widgets' values to update the plots
@pn.depends(sel_month=month_slider_v_avg,
            depth=depth_range_v_avg, 
            col_range=colorbar_slider_v_avg,
            discrete_colors=discrete_colorbar_toggle_v_avg,
            bin_count=colorbins_slider_v_avg)
def update_plot_v_c(sel_month, depth, col_range, discrete_colors, bin_count):
    plot = hv.QuadMesh(ds_v_c.loc[sel_month[0]:sel_month[1], depth[0]:depth[1]].mean(['month','st_ocean']), 
                       kdims=["xu_ocean", "yu_ocean"], vdims=hv.Dimension('u', range=(-col_range, col_range)))
    plot.opts(cmap=plt.cm.get_cmap(cmap_v_avg, bin_count) if discrete_colors else cmap_v_avg)
    plot.opts(title=f'CTRL-9091 V-velocity between {depth[0]:d}–{depth[1]:d} (m) depth, '
              f'{calendar.month_name[np.int(sel_month[0])]}–{calendar.month_name[np.int(sel_month[1])]}', 
              framewise=True, colorbar=True, 
              clabel="m/s", width=900, height=250, tools=['hover'], ylabel='Latitude (°N)', xlabel='Longitude (°E)', 
              bgcolor='Silver', toolbar="above", hooks=[set_inactive_tool], active_tools=['box_zoom'])
        
    return plot

# Watch for changes on the widgets' values to update the plots
@pn.depends(sel_month=month_slider_v_avg,
            col_range=colorbar_slider_v_avg,
            discrete_colors=discrete_colorbar_toggle_v_avg,
            bin_count=colorbins_slider_v_avg)
def update_cross_v_c(x, sel_month, col_range, discrete_colors, bin_count):
    longitud = closest_lon_u(x or (initial_lon))
    
    plot = hv.QuadMesh(ds_v_c.sel(month=slice(sel_month[0],sel_month[1]), xu_ocean=longitud).mean('month'), 
                       kdims=["yu_ocean", "st_ocean"], 
                       vdims=hv.Dimension('u', range=(-col_range, col_range)))
    plot.opts(cmap=plt.cm.get_cmap(cmap_v_avg, bin_count) if discrete_colors else cmap_v_avg)
    plot.opts(title='CTRL-9091 V-velocity cross-section at %d°E'% (longitud), framewise=True, invert_yaxis=True, 
              colorbar=True, clabel="m/s", bgcolor='Silver', tools=['hover'], width=900, height=450,
              ylabel='Depth (m)', xlabel='Latitude (°N)', toolbar="above", hooks=[set_inactive_tool],
              active_tools=['box_zoom'])
    return plot


# DIFFERENCES
# Watch for changes on the widgets' values to update the plots
@pn.depends(sel_month=month_slider_v_avg,
            depth=depth_range_v_avg,  
            col_range=colorbar_slider_v_dif,
            discrete_colors=discrete_colorbar_toggle_v_avg,
            bin_count=colorbins_slider_v_avg)
def update_plot_v_dif(sel_month, depth, col_range, discrete_colors, bin_count):
    plot = hv.QuadMesh(ds_v_dif.loc[sel_month[0]:sel_month[1], depth[0]:depth[1]].mean(['month','st_ocean']), 
                       kdims=["xu_ocean", "yu_ocean"], vdims=hv.Dimension('u', range=(-col_range, col_range)))
    plot.opts(cmap=plt.cm.get_cmap(cmap_v_dif, bin_count) if discrete_colors else cmap_v_dif)
    plot.opts(title=f'DiFFERENCES in V-velocity between {depth[0]:d}–{depth[1]:d} (m) depth, '
              f'{calendar.month_name[np.int(sel_month[0])]}–{calendar.month_name[np.int(sel_month[1])]}', 
              framewise=True, colorbar=True, 
              clabel="m/s", width=900, height=250, tools=['hover'], ylabel='Latitude (°N)', xlabel='Longitude (°E)', 
              bgcolor='Silver', toolbar="above", hooks=[set_inactive_tool], active_tools=['box_zoom'])
        
    return plot

# Watch for changes on the widgets' values to update the plots
@pn.depends(sel_month=month_slider_v_avg, 
            col_range=colorbar_slider_v_dif,
            discrete_colors=discrete_colorbar_toggle_v_avg,
            bin_count=colorbins_slider_v_avg)
def update_cross_v_dif(x, sel_month, col_range, discrete_colors, bin_count):
    longitud = closest_lon_u(x or (initial_lon))
    
    plot = hv.QuadMesh(ds_v_dif.sel(month=slice(sel_month[0],sel_month[1]), xu_ocean=longitud).mean('month'), 
                       kdims=["yu_ocean", "st_ocean"], 
                       vdims=hv.Dimension('u', range=(-col_range, col_range)))
    plot.opts(cmap=plt.cm.get_cmap(cmap_v_dif, bin_count) if discrete_colors else cmap_v_dif)
    plot.opts(title='DiFFERENCES in V-velocity cross-section at %d°E'% (longitud), framewise=True, invert_yaxis=True, 
              colorbar=True, clabel="m/s", bgcolor='Silver', tools=['hover'], width=900, height=450,
              ylabel='Depth (m)', xlabel='Latitude (°N)', toolbar="above", hooks=[set_inactive_tool],
              active_tools=['box_zoom'])
    return plot

# ~~~~~ ~~~~~ ~~~~~ ~~~~~ ~~~~~ ^_^ ~~~~~ ~~~~~ ~~~~~ ~~~~~ ~~~~~
# LONGITUDE LINE

vline_v_avg = hv.DynamicMap(lambda x: hv.VLine(x or initial_lon), streams=[posx_v_avg])
vline_v_avg.opts(color="RoyalBlue", line_dash='dashed')

# DEPTH LINES
hline_v_avg_upper = hv.DynamicMap(update_hline_v_avg_upper)
hline_v_avg_upper.opts(color="RoyalBlue", line_dash='dashed')

hline_v_avg_lower = hv.DynamicMap(update_hline_v_avg_lower)
hline_v_avg_lower.opts(color="RoyalBlue", line_dash='dashed')

# U ANTARCTIC PLOT
plot_v_avg = hv.DynamicMap(update_plot_v_avg)
# U CROSS-SECTION
cross_v_avg = hv.DynamicMap(update_cross_v_avg, streams=[posx_v_avg])

# CTRL U ANTARCTIC PLOT
plot_v_c = hv.DynamicMap(update_plot_v_c)
# CTRL U CROSS-SECTION
cross_v_c = hv.DynamicMap(update_cross_v_c, streams=[posx_v_avg])

# DIFFERENCES U ANTARCTIC PLOT
plot_v_dif = hv.DynamicMap(update_plot_v_dif)
# DIFFERENCES U CROSS-SECTION
cross_v_dif = hv.DynamicMap(update_cross_v_dif, streams=[posx_v_avg])

# ~~~~~ ~~~~~ ~~~~~ ~~~~~ ~~~~~ ^_^ ~~~~~ ~~~~~ ~~~~~ ~~~~~ ~~~~~
# Build the layout
pn.Column(pn.Row(pn.Spacer(width=50), depth_range_v_avg, month_slider_v_avg), 
          pn.Row(pn.Spacer(width=50), colorbar_slider_v_avg, discrete_colorbar_toggle_v_avg, 
                 pn.Spacer(width=2), colorbins_slider_v_avg),
          pn.Spacer(height=5),
          plot_v_avg * vline_v_avg,
          cross_v_avg * hline_v_avg_upper * hline_v_avg_lower,
          plot_v_c * vline_v_avg,
          cross_v_c * hline_v_avg_upper * hline_v_avg_lower,
          plot_v_dif * vline_v_avg,
          cross_v_dif * hline_v_avg_upper * hline_v_avg_lower,
          pn.Row(pn.Spacer(width=50), colorbar_slider_v_dif))

# ZOCO

In [ ]:
def antarctica(ax, max_lat, show_details):
    """
    Creates a stereographic, round plot of Antarctica
    
    Parameters:
    -----------
        ax: axis on which to plot Antarctica
        max_lat: negative integer with the maximum latitude to show (i.e. -°S)
        show_details: boolean indicating whether to plot the land, ocean & grids
    
    Returns:
    --------
        ax: axis with Antarctica plotted 
    """
    
    # Limit the map to max_lat degrees latitude and below
    ax.set_extent([-180, 180, -90, max_lat], ccrs.PlateCarree())

    if (show_details):
        gl = ax.gridlines(color='DarkSlateGray', linestyle='--')
        # Show gridlines: meridians every 45°, parallels every 15°
        gl.xlocator = mticker.FixedLocator(np.arange(-180, 181, 45))
        gl.ylocator =  mticker.FixedLocator(np.arange(-80, -34, 15))
 
        #ax.add_feature(cfeature.LAND, zorder=1)
        #ax.add_feature(ice_shelves)
        #ax.add_feature(cfeature.OCEAN)

    # Compute a circle in axes coordinates to use as the map's boundary
    theta = np.linspace(0, 2*np.pi, 100)
    center, radius = [0.5, 0.5], 0.5
    verts = np.vstack([np.sin(theta), np.cos(theta)]).T
    circle = mpath.Path(verts * radius + center)

    ax.set_boundary(circle, transform=ax.transAxes)
    
    return ax

In [ ]:
def colorea (plot, axis, units):
    """
    Creates a colorbar for the given axis, with appropriate units & font sizes
    
    Parameters:
    -----------
        plot: plot in which to create the colorbar
        axis: axis in which to create the colorbar
        units: string with units of the colorbar
    """
    
    cb = plt.colorbar(plot, ax=axis, shrink=0.7)
    cb.ax.tick_params(labelsize=20)
    cb.set_label(units, fontsize=20)

In [ ]:
# Calculate the zonal transport

# **kwarg min_count is to retain NaN values, because otherwise sum(NaNs)=0 (in which universe!)
pi_psi = dso.tx_trans.mean('time').sum('st_ocean', min_count=10)
pi_psi = pi_psi.cumsum('yt_ocean').where(~np.isnan(pi_psi))
pi_psi = pi_psi / 1000000000 # Convert to Sv

pi_psi_c = dso_c.tx_trans.mean('time').sum('st_ocean', min_count=10)
pi_psi_c = pi_psi_c.cumsum('yt_ocean').where(~np.isnan(pi_psi_c))
pi_psi_c = pi_psi_c / 1000000000 # Convert to Sv

pi_psi_dif = pi_psi - pi_psi_c

In [ ]:
import matplotlib.path as mpath
import matplotlib.ticker as mticker
from palettable.cartocolors.qualitative import Prism_10_r

# Widget to select the SCALE for the contours
scale_slider_u = pn.widgets.IntSlider(name='Number of Levels', start=4, end=28, step=4, 
                                             value=16, width=150)
# Widget to show/hide the contour lines
show_contours = pn.widgets.Toggle(name='Show Contour Lines', width=150, value=True)

# EXPERIMENT's — PLOT THE ACC TRANSPORT

# Plots' parameters
central_lon = -67.5
mapa = Prism_10_r.mpl_colormap
max_lat = -45
show_details = True
units = "Transport (Sv)"

@pn.depends(sel_scale=scale_slider_u,
            show_lines=show_contours)  
def plot_zoco(sel_scale, show_lines):
    # Define the figure
    plt.figure(figsize=(10,30))

    # CTRL
    ax1 = plt.subplot(3, 1, 1, projection=ccrs.SouthPolarStereo(central_longitude=central_lon))
    ax1 = antarctica(ax1, max_lat, show_details)
    plot1 = ax1.contourf(pi_psi_c.xu_ocean, pi_psi_c.yt_ocean.sel(yt_ocean=slice(-90, max_lat)), 
                         pi_psi_c.sel(yt_ocean=slice(-90, max_lat)), levels=sel_scale,
                         cmap=Prism_10_r.mpl_colormap, vmin=-30, vmax=192, transform=ccrs.PlateCarree(), zorder=0)
    colorea(plot1, ax1, units)
    if show_lines:
        plot1 = ax1.contour(pi_psi_c.xu_ocean, pi_psi_c.yt_ocean.sel(yt_ocean=slice(-90, max_lat)), 
                             pi_psi_c.sel(yt_ocean=slice(-90, max_lat)), levels=sel_scale,
                             colors="k", linewidths=0.5, vmin=-30, vmax=192, transform=ccrs.PlateCarree(), zorder=0)
        ax1.clabel(plot1, inline = 1, fmt = '%1.1f')
    plt.title('Ctrl', fontsize=40, pad=20)
    
    # EXPERIMENT
    ax2 = plt.subplot(3, 1, 2, projection=ccrs.SouthPolarStereo(central_longitude=central_lon))
    ax2 = antarctica(ax2, max_lat, show_details)
    plot2 = ax2.contourf(pi_psi.xu_ocean, pi_psi.yt_ocean.sel(yt_ocean=slice(-90, max_lat)), 
                         pi_psi.sel(yt_ocean=slice(-90, max_lat)), levels=sel_scale,
                         cmap=Prism_10_r.mpl_colormap, vmin=-30, vmax=192, transform=ccrs.PlateCarree(), zorder=0)
    colorea(plot2, ax2, units)
    if show_lines:
        plot2 = ax2.contour(pi_psi.xu_ocean, pi_psi.yt_ocean.sel(yt_ocean=slice(-90, max_lat)), 
                             pi_psi.sel(yt_ocean=slice(-90, max_lat)), levels=sel_scale,
                             colors="k", linewidths=0.5, vmin=-30, vmax=192, transform=ccrs.PlateCarree(), zorder=0)
        ax2.clabel(plot2, inline = 1, fmt = '%1.1f')
    plt.title('Exp.', fontsize=40, pad=20)
    
    # DiFFERENCES
    ax3 = plt.subplot(3, 1, 3, projection=ccrs.SouthPolarStereo(central_longitude=central_lon))
    ax3 = antarctica(ax3, max_lat, show_details)
    plot3 = ax3.contourf(pi_psi_dif.xu_ocean, pi_psi_dif.yt_ocean.sel(yt_ocean=slice(-90, max_lat)), 
                         pi_psi_dif.sel(yt_ocean=slice(-90, max_lat)), levels=sel_scale,
                         cmap="RdBu_r", vmin=-8, vmax=8, transform=ccrs.PlateCarree(), zorder=0)
    colorea(plot3, ax3, units)
    if show_lines:
        plot3 = ax3.contour(pi_psi_dif.xu_ocean, pi_psi_dif.yt_ocean.sel(yt_ocean=slice(-90, max_lat)), 
                             pi_psi_dif.sel(yt_ocean=slice(-90, max_lat)), levels=sel_scale,
                             colors="k", linewidths=0.5, transform=ccrs.PlateCarree(), zorder=0)
        ax3.clabel(plot3, inline = 1, fmt = '%1.1f')
    plt.title('Differences', fontsize=40, pad=20)
    
    # ~~~~~ ~~~~~ ~~~~~ ~~~~~ ~~~~~ ^_^ ~~~~~ ~~~~~ ~~~~~ ~~~~~ ~~~~~
    curr_fig=plt.gcf()  
    # Since curr_fig.tight_layout() doesn't work for some unholy reason, we're forced to manually 
    # destroy the margins
    curr_fig.subplots_adjust(left=0, right=1, top=0.95, bottom=0.05)
    plt.close(curr_fig) 
    return curr_fig
 
# Show resulting figure
pn.Column(pn.Row(scale_slider_u, show_contours),
          plot_zoco)

# TEMPERATURE

In [ ]:
# Temperature averaged through time
ds_temp = dsm.temp.loc[:,:,-78:-45,:].mean('time')
ds_temp -= 273.15

# Ctrl Temperature averaged through time
ds_temp_c = dsm_c.temp.loc[:,:,-78:-45,:].mean('time')
ds_temp_c -= 273.15

# Differences
ds_temp_dif = ds_temp - ds_temp_c
#ds_temp_dif.compute()

In [ ]:
hv.extension('bokeh')

colorbar_range_temp = (-2, 12)
cmap_temp = "cmo.thermal"
initial_lon = -68.5

# Stream to follow the pointer over the upper plot (and then update the lower plot's cross-section)
posx_temp = hv.streams.PointerX()

# ~~~~~ ~~~~~ ~~~~~ ~~~~~ ~~~~~ ^_^ ~~~~~ ~~~~~ ~~~~~ ~~~~~ ~~~~~
# WIDGETS

# Widget to select the DEPTH
depth_slider_temp = pn.widgets.DiscreteSlider(name='Depth', options=ds_temp.st_ocean.values.tolist(), 
                                             value=ds_temp.st_ocean.values[0], width=500, formatter='%.2f (m)')

# Widget to select the COLORBAR RANGE
colorbar_range_slider_temp = pn.widgets.RangeSlider(name='Colorbar Range', start=colorbar_range_temp[0], 
                                                    end=colorbar_range_temp[1],
                                                    value=colorbar_range_temp, step=0.25, width=500)

# Widget to activate a DISCRETE COLORBAR (instead of a continuous one)
discrete_colorbar_toggle_temp = pn.widgets.Toggle(name='Show Discrete Colors', width=150)

# Widget to activate a HIGH-CONTRAST COLORBAR (instead of a continuous one)
contrast_colorbar_toggle_temp = pn.widgets.Toggle(name='Show High-Contrast Colors', width=150)

# Widget to select the NUMBER of BINS of the colorbar
colorbins_slider_temp = pn.widgets.IntSlider(name='Colorbar # Bins', start=5, end=30, step=1, value=15,
                                            width=200)

# ~~~~~ ~~~~~ ~~~~~ ~~~~~ ~~~~~ ^_^ ~~~~~ ~~~~~ ~~~~~ ~~~~~ ~~~~~
# WIDGETS's LINKS

# Attach a link to the widget value, so it automatically refreshes the vertical, 
# longitude line upon change
@pn.depends(high_contrast=contrast_colorbar_toggle_temp.param.value)
def update_vline_temp(x, high_contrast):
    line = hv.VLine(x or initial_lon)
    line.opts(color="Black" if high_contrast else "MediumSeaGreen", line_dash='dashed')
    return line

# Attach a link to the widget value, so it automatically refreshes the horizontal, 
# depth line upon change
@pn.depends(depth=depth_slider_temp.param.value,
            high_contrast=contrast_colorbar_toggle_temp.param.value)
def update_hline_temp(depth, high_contrast):
    line = hv.HLine(depth)
    line.opts(color="Black" if high_contrast else "MediumSeaGreen", line_dash='dashed')
    return line

# Watch for changes on the widgets' values to update the plots
@pn.depends(depth=depth_slider_temp.param.value, 
            col_range=colorbar_range_slider_temp.param.value,
            discrete_colors=discrete_colorbar_toggle_temp.param.value,
            bin_count=colorbins_slider_temp.param.value,
            high_contrast=contrast_colorbar_toggle_temp.param.value)
def update_plot_temp(depth, col_range, discrete_colors, bin_count, high_contrast):
    plot = hv.QuadMesh(ds_temp.loc[depth], kdims=["xt_ocean", "yt_ocean"], 
                       vdims=hv.Dimension('temp', range=col_range))
    if high_contrast:
        plot.opts(cmap=Prism_8.hex_colors if discrete_colors else Prism_8.mpl_colormap)#brewer['Spectral'][10]
    else:
        plot.opts(cmap=plt.cm.get_cmap(cmap_temp, bin_count) if discrete_colors else cmap_temp)
    plot.opts(title='Temperature at %.2f(m) depth'% (depth), framewise=True, colorbar=True, 
              clabel="°C", width=900, height=250, tools=['hover'], yaxis=None, xlabel='Longitude (°E)', 
              bgcolor='Silver', toolbar="above")
    return plot

# Watch for changes on the widgets' values to update the plots
@pn.depends(depth=depth_slider_temp.param.value, 
            col_range=colorbar_range_slider_temp.param.value,
            discrete_colors=discrete_colorbar_toggle_temp.param.value,
            bin_count=colorbins_slider_temp.param.value,
            high_contrast=contrast_colorbar_toggle_temp.param.value)
def update_cross_temp(x, depth, col_range, discrete_colors, bin_count, high_contrast):
    longitud = closest_lon(x or initial_lon)
    
    plot = hv.QuadMesh(ds_temp.sel(xt_ocean=longitud), 
                       kdims=["yt_ocean", "st_ocean"], 
                       vdims=hv.Dimension('temp', range=col_range))
    if high_contrast:
        plot.opts(cmap=Prism_8.hex_colors if discrete_colors else Prism_8.mpl_colormap)
    else:
        plot.opts(cmap=plt.cm.get_cmap(cmap_temp, bin_count) if discrete_colors else cmap_temp)
    plot.opts(title='Temperature cross-section at %.1f°E'% (longitud), framewise=True, invert_yaxis=True, 
              colorbar=True, clabel="°C", bgcolor='Silver', tools=['hover'], width=900, height=450,
              ylabel='Depth (m)', xlabel='Latitude (°N)', toolbar="above")
    return plot

# ~~~~~ ~~~~~ ~~~~~ ~~~~~ ~~~~~ ^_^ ~~~~~ ~~~~~ ~~~~~ ~~~~~ ~~~~~
# LONGITUDE LINE
vline_temp = hv.DynamicMap(update_vline_temp, streams=[posx_temp])

# DEPTH LINE
hline_temp = hv.DynamicMap(update_hline_temp)

# TEMPERATURE ANTARCTIC PLOT
plot_temp = hv.DynamicMap(update_plot_temp)

# TEMPERATURE CROSS-SECTION
cross_temp = hv.DynamicMap(update_cross_temp, streams=[posx_temp])

# ~~~~~ ~~~~~ ~~~~~ ~~~~~ ~~~~~ ^_^ ~~~~~ ~~~~~ ~~~~~ ~~~~~ ~~~~~
# Build the layout, using Spacers as padding amongst elements 
pn.Column(pn.Row(pn.Spacer(width=50), depth_slider_temp, discrete_colorbar_toggle_temp, contrast_colorbar_toggle_temp), 
          pn.Row(pn.Spacer(width=50), colorbar_range_slider_temp, colorbins_slider_temp),
          pn.Spacer(height=5),
          plot_temp * vline_temp,
          cross_temp * hline_temp)

# <font color=gray>DiFFERENCES in TEMPERATURE</font>

In [ ]:
hv.extension('bokeh')

colorbar_range_temp_dif = (-2, 2)
cmap_temp_dif = "RdBu_r"
initial_lon = -68.5

# Stream to follow the pointer over the upper plot (and then update the lower plot's cross-section)
posx_temp_dif = hv.streams.PointerX()

# ~~~~~ ~~~~~ ~~~~~ ~~~~~ ~~~~~ ^_^ ~~~~~ ~~~~~ ~~~~~ ~~~~~ ~~~~~
# WIDGETS

# Widget to select the DEPTH
depth_slider_temp_dif = pn.widgets.DiscreteSlider(name='Depth', options=ds_temp_dif.st_ocean.values.tolist(), 
                                             value=ds_temp_dif.st_ocean.values[0], width=500, formatter='%.2f (m)')

# Widget to select the COLORBAR RANGE
colorbar_slider_temp_dif = pn.widgets.FloatSlider(name='Colorbar Range', start=0, end=colorbar_range_temp_dif[1], 
                                                  step=0.1, value=1, width=500)

# Widget to activate a DISCRETE COLORBAR (instead of a continuous one)
discrete_colorbar_toggle_temp_dif = pn.widgets.Toggle(name='Show Discrete Colors', width=150)

# Widget to select the NUMBER of BINS of the colorbar
colorbins_slider_temp_dif = pn.widgets.IntSlider(name='Colorbar # Bins', start=5, end=30, step=1, value=15,
                                            width=200)

# ~~~~~ ~~~~~ ~~~~~ ~~~~~ ~~~~~ ^_^ ~~~~~ ~~~~~ ~~~~~ ~~~~~ ~~~~~
# WIDGETS's LINKS

# Attach a link to the widget value, so it automatically refreshes the horizontal, 
# depth line upon change
@pn.depends(depth=depth_slider_temp_dif.param.value)
def update_hline_temp_dif(depth):
    return hv.HLine(depth)

# Watch for changes on the widgets' values to update the plots
@pn.depends(depth=depth_slider_temp_dif.param.value, 
            col_range=colorbar_slider_temp_dif.param.value,
            discrete_colors=discrete_colorbar_toggle_temp_dif.param.value,
            bin_count=colorbins_slider_temp_dif.param.value)
def update_plot_temp_dif(depth, col_range, discrete_colors, bin_count):
    plot = hv.QuadMesh(ds_temp_dif.loc[depth], kdims=["xt_ocean", "yt_ocean"], 
                       vdims=hv.Dimension('temp', range=(-col_range, col_range)))
    plot.opts(cmap=plt.cm.get_cmap(cmap_temp_dif, bin_count) if discrete_colors else cmap_temp_dif)
    plot.opts(title='Differences in Temperature at %.2f(m) depth'% (depth), framewise=True, colorbar=True, 
              clabel="°C", width=900, height=250, tools=['hover'], yaxis=None, xlabel='Longitude (°E)', 
              bgcolor='Silver', toolbar="above")
    return plot

# Watch for changes on the widgets' values to update the plots
@pn.depends(depth=depth_slider_temp_dif.param.value, 
            col_range=colorbar_slider_temp_dif.param.value,
            discrete_colors=discrete_colorbar_toggle_temp_dif.param.value,
            bin_count=colorbins_slider_temp_dif.param.value)
def update_cross_temp_dif(x, depth, col_range, discrete_colors, bin_count):
    longitud = closest_lon(x or (initial_lon))
    
    plot = hv.QuadMesh(ds_temp_dif.sel(xt_ocean=longitud), 
                       kdims=["yt_ocean", "st_ocean"], 
                       vdims=hv.Dimension('temp', range=(-col_range, col_range)))
    plot.opts(cmap=plt.cm.get_cmap(cmap_temp_dif, bin_count) if discrete_colors else cmap_temp_dif)
    plot.opts(title='Differences in Temperature: cross-section at %.1f°E'% (longitud), framewise=True, invert_yaxis=True, 
              colorbar=True, clabel="°C", bgcolor='Silver', tools=['hover'], width=900, height=450,
              ylabel='Depth (m)', xlabel='Latitude (°N)', toolbar="above")
    return plot

# ~~~~~ ~~~~~ ~~~~~ ~~~~~ ~~~~~ ^_^ ~~~~~ ~~~~~ ~~~~~ ~~~~~ ~~~~~
# LONGITUDE LINE

vline_temp_dif = hv.DynamicMap(lambda x: hv.VLine(x or initial_lon), streams=[posx_temp_dif])
vline_temp_dif.opts(color="MediumSeaGreen", line_dash='dashed')

# DEPTH LINE
hline_temp_dif = hv.DynamicMap(update_hline_temp_dif)
hline_temp_dif.opts(color="MediumSeaGreen", line_dash='dashed')

# U ANTARCTIC PLOT
plot_temp_dif = hv.DynamicMap(update_plot_temp_dif)

# U CROSS-SECTION
cross_temp_dif = hv.DynamicMap(update_cross_temp_dif, streams=[posx_temp_dif])

# ~~~~~ ~~~~~ ~~~~~ ~~~~~ ~~~~~ ^_^ ~~~~~ ~~~~~ ~~~~~ ~~~~~ ~~~~~
# Build the layout, using Spacers as padding amongst elements 
pn.Column(pn.Row(pn.Spacer(width=50), depth_slider_temp_dif, discrete_colorbar_toggle_temp_dif), 
          pn.Row(pn.Spacer(width=50), colorbar_slider_temp_dif, colorbins_slider_temp_dif),
          pn.Spacer(height=5),
          plot_temp_dif * vline_temp_dif,
          cross_temp_dif * hline_temp_dif)

# SALINITY

In [ ]:
# Salt averaged through time
ds_salt = dsm.salt.loc[:,:,-78:-45,:].mean('time')

# Ctrl Salt averaged through time
ds_salt_c = dsm_c.salt.loc[:,:,-78:-45,:].mean('time')

# Differences
ds_salt_dif = ds_salt - ds_salt_c
#ds_salt_dif.compute()

In [ ]:
hv.extension('bokeh')

colorbar_range_salt = (33, 35)
cmap_salt = "cmo.haline"
initial_lon = -68.5

# Stream to follow the pointer over the upper plot (and then update the lower plot's cross-section)
posx_salt = hv.streams.PointerX()

# ~~~~~ ~~~~~ ~~~~~ ~~~~~ ~~~~~ ^_^ ~~~~~ ~~~~~ ~~~~~ ~~~~~ ~~~~~
# WIDGETS

# Widget to select the DEPTH
depth_slider_salt = pn.widgets.DiscreteSlider(name='Depth', options=ds_salt.st_ocean.values.tolist(), 
                                             value=ds_salt.st_ocean.values[0], width=500, formatter='%.2f (m)')

# Widget to select the COLORBAR RANGE
colorbar_range_slider_salt = pn.widgets.RangeSlider(name='Colorbar Range', start=colorbar_range_salt[0], 
                                                    end=colorbar_range_salt[1],
                                                    value=colorbar_range_salt, step=0.05, width=500)

# Widget to activate a DISCRETE COLORBAR (instead of a continuous one)
discrete_colorbar_toggle_salt = pn.widgets.Toggle(name='Show Discrete Colors', width=150)

# Widget to activate a HIGH-CONTRAST COLORBAR (instead of a continuous one)
contrast_colorbar_toggle_salt = pn.widgets.Toggle(name='Show High-Contrast Colors', width=150)

# Widget to select the NUMBER of BINS of the colorbar
colorbins_slider_salt = pn.widgets.IntSlider(name='Colorbar # Bins', start=5, end=30, step=1, value=15,
                                            width=200)

# ~~~~~ ~~~~~ ~~~~~ ~~~~~ ~~~~~ ^_^ ~~~~~ ~~~~~ ~~~~~ ~~~~~ ~~~~~
# WIDGETS's LINKS

# Attach a link to the widget value, so it automatically refreshes the vertical, 
# longitude line upon change
@pn.depends(high_contrast=contrast_colorbar_toggle_salt.param.value)
def update_vline_salt(x, high_contrast):
    line = hv.VLine(x or initial_lon)
    line.opts(color="Black" if high_contrast else "Crimson", line_dash='dashed')
    return line

# Attach a link to the widget value, so it automatically refreshes the horizontal, 
# depth line upon change
@pn.depends(depth=depth_slider_salt.param.value,
            high_contrast=contrast_colorbar_toggle_salt.param.value)
def update_hline_salt(depth, high_contrast):
    line = hv.HLine(depth)
    line.opts(color="Black" if high_contrast else "Crimson", line_dash='dashed')
    return line

# Watch for changes on the widgets' values to update the plots
@pn.depends(depth=depth_slider_salt.param.value, 
            col_range=colorbar_range_slider_salt.param.value,
            discrete_colors=discrete_colorbar_toggle_salt.param.value,
            bin_count=colorbins_slider_salt.param.value,
            high_contrast=contrast_colorbar_toggle_salt.param.value)
def update_plot_salt(depth, col_range, discrete_colors, bin_count, high_contrast):
    plot = hv.QuadMesh(ds_salt.loc[depth], kdims=["xt_ocean", "yt_ocean"], 
                       vdims=hv.Dimension('temp', range=col_range))
    if high_contrast:
        plot.opts(cmap=Prism_8.hex_colors if discrete_colors else Prism_8.mpl_colormap)
    else:
        plot.opts(cmap=plt.cm.get_cmap(cmap_salt, bin_count) if discrete_colors else cmap_salt)
    plot.opts(title='Salinity at %.2f(m) depth'% (depth), framewise=True, colorbar=True, 
              clabel="psu", width=900, height=250, tools=['hover'], yaxis=None, xlabel='Longitude (°E)', 
              bgcolor='Silver', toolbar="above")
    return plot

# Watch for changes on the widgets' values to update the plots
@pn.depends(depth=depth_slider_salt.param.value, 
            col_range=colorbar_range_slider_salt.param.value,
            discrete_colors=discrete_colorbar_toggle_salt.param.value,
            bin_count=colorbins_slider_salt.param.value,
            high_contrast=contrast_colorbar_toggle_salt.param.value)
def update_cross_salt(x, depth, col_range, discrete_colors, bin_count, high_contrast):
    longitud = closest_lon(x or (initial_lon))
    
    plot = hv.QuadMesh(ds_salt.sel(xt_ocean=longitud), 
                       kdims=["yt_ocean", "st_ocean"], 
                       vdims=hv.Dimension('temp', range=col_range))
    if high_contrast:
        plot.opts(cmap=Prism_8.hex_colors if discrete_colors else Prism_8.mpl_colormap)
    else:
        plot.opts(cmap=plt.cm.get_cmap(cmap_salt, bin_count) if discrete_colors else cmap_salt)
    plot.opts(title='Salinity cross-section at %.1f°E'% (longitud), framewise=True, invert_yaxis=True, 
              colorbar=True, clabel="psu", bgcolor='Silver', tools=['hover'], width=900, height=450,
              ylabel='Depth (m)', xlabel='Latitude (°N)', toolbar="above")
    return plot

# ~~~~~ ~~~~~ ~~~~~ ~~~~~ ~~~~~ ^_^ ~~~~~ ~~~~~ ~~~~~ ~~~~~ ~~~~~
# LONGITUDE LINE

vline_salt = hv.DynamicMap(update_vline_salt, streams=[posx_salt])

# DEPTH LINE
hline_salt = hv.DynamicMap(update_hline_salt)

# U ANTARCTIC PLOT
plot_salt = hv.DynamicMap(update_plot_salt)

# U CROSS-SECTION
cross_salt = hv.DynamicMap(update_cross_salt, streams=[posx_salt])

# ~~~~~ ~~~~~ ~~~~~ ~~~~~ ~~~~~ ^_^ ~~~~~ ~~~~~ ~~~~~ ~~~~~ ~~~~~
# Build the layout, using Spacers as padding amongst elements 
pn.Column(pn.Row(pn.Spacer(width=50), depth_slider_salt, discrete_colorbar_toggle_salt, contrast_colorbar_toggle_salt), 
          pn.Row(pn.Spacer(width=50), colorbar_range_slider_salt, colorbins_slider_salt),
          pn.Spacer(height=5),
          plot_salt * vline_salt,
          cross_salt * hline_salt)

# <font color=gray>DiFFERENCES in SALINITY</font>

In [ ]:
hv.extension('bokeh')

colorbar_range_salt_dif = (-0.5, 0.5)
cmap_salt_dif = "RdBu_r"
initial_lon = -68.5

# Stream to follow the pointer over the upper plot (and then update the lower plot's cross-section)
posx_salt_dif = hv.streams.PointerX()

# ~~~~~ ~~~~~ ~~~~~ ~~~~~ ~~~~~ ^_^ ~~~~~ ~~~~~ ~~~~~ ~~~~~ ~~~~~
# WIDGETS

# Widget to select the DEPTH
depth_slider_salt_dif = pn.widgets.DiscreteSlider(name='Depth', options=ds_salt_dif.st_ocean.values.tolist(), 
                                             value=ds_salt_dif.st_ocean.values[0], width=500, formatter='%.2f (m)')

# Widget to select the COLORBAR RANGE
colorbar_slider_salt_dif = pn.widgets.FloatSlider(name='Colorbar Range', start=0, end=colorbar_range_salt_dif[1], 
                                                  step=0.05, value=0.25, width=500)

# Widget to activate a DISCRETE COLORBAR (instead of a continuous one)
discrete_colorbar_toggle_salt_dif = pn.widgets.Toggle(name='Show Discrete Colors', width=150)

# Widget to select the NUMBER of BINS of the colorbar
colorbins_slider_salt_dif = pn.widgets.IntSlider(name='Colorbar # Bins', start=5, end=30, step=1, value=15,
                                            width=200)

# ~~~~~ ~~~~~ ~~~~~ ~~~~~ ~~~~~ ^_^ ~~~~~ ~~~~~ ~~~~~ ~~~~~ ~~~~~
# WIDGETS's LINKS

# Attach a link to the widget value, so it automatically refreshes the horizontal, 
# depth line upon change
@pn.depends(depth=depth_slider_salt_dif.param.value)
def update_hline_salt_dif(depth):
    return hv.HLine(depth)

# Watch for changes on the widgets' values to update the plots
@pn.depends(depth=depth_slider_salt_dif.param.value, 
            col_range=colorbar_slider_salt_dif.param.value,
            discrete_colors=discrete_colorbar_toggle_salt_dif.param.value,
            bin_count=colorbins_slider_salt_dif.param.value)
def update_plot_salt_dif(depth, col_range, discrete_colors, bin_count):
    plot = hv.QuadMesh(ds_salt_dif.loc[depth], kdims=["xt_ocean", "yt_ocean"], 
                       vdims=hv.Dimension('psu', range=(-col_range, col_range)))
    plot.opts(cmap=plt.cm.get_cmap(cmap_salt_dif, bin_count) if discrete_colors else cmap_salt_dif)
    plot.opts(title='Differences in Salinity at %.2f(m) depth'% (depth), framewise=True, colorbar=True, 
              clabel="psu", width=900, height=250, tools=['hover'], yaxis=None, xlabel='Longitude (°E)', 
              bgcolor='Silver', toolbar="above")
    return plot

# Watch for changes on the widgets' values to update the plots
@pn.depends(depth=depth_slider_salt_dif.param.value, 
            col_range=colorbar_slider_salt_dif.param.value,
            discrete_colors=discrete_colorbar_toggle_salt_dif.param.value,
            bin_count=colorbins_slider_salt_dif.param.value)
def update_cross_salt_dif(x, depth, col_range, discrete_colors, bin_count):
    longitud = closest_lon(x or (initial_lon))
    
    plot = hv.QuadMesh(ds_salt_dif.sel(xt_ocean=longitud), 
                       kdims=["yt_ocean", "st_ocean"], 
                       vdims=hv.Dimension('psu', range=(-col_range, col_range)))
    plot.opts(cmap=plt.cm.get_cmap(cmap_salt_dif, bin_count) if discrete_colors else cmap_salt_dif)
    plot.opts(title='Differences in Salinity: cross-section at %.1f°E'% (longitud), framewise=True, invert_yaxis=True, 
              colorbar=True, clabel="psu", bgcolor='Silver', tools=['hover'], width=900, height=450,
              ylabel='Depth (m)', xlabel='Latitude (°N)', toolbar="above")
    return plot

# ~~~~~ ~~~~~ ~~~~~ ~~~~~ ~~~~~ ^_^ ~~~~~ ~~~~~ ~~~~~ ~~~~~ ~~~~~
# LONGITUDE LINE

vline_salt_dif = hv.DynamicMap(lambda x: hv.VLine(x or initial_lon), streams=[posx_salt_dif])
vline_salt_dif.opts(color="MediumSeaGreen", line_dash='dashed')

# DEPTH LINE
hline_salt_dif = hv.DynamicMap(update_hline_salt_dif)
hline_salt_dif.opts(color="MediumSeaGreen", line_dash='dashed')

# U ANTARCTIC PLOT
plot_salt_dif = hv.DynamicMap(update_plot_salt_dif)

# U CROSS-SECTION
cross_salt_dif = hv.DynamicMap(update_cross_salt_dif, streams=[posx_salt_dif])

# ~~~~~ ~~~~~ ~~~~~ ~~~~~ ~~~~~ ^_^ ~~~~~ ~~~~~ ~~~~~ ~~~~~ ~~~~~
# Build the layout, using Spacers as padding amongst elements 
pn.Column(pn.Row(pn.Spacer(width=50), depth_slider_salt_dif, discrete_colorbar_toggle_salt_dif), 
          pn.Row(pn.Spacer(width=50), colorbar_slider_salt_dif, colorbins_slider_salt_dif),
          pn.Spacer(height=5),
          plot_salt_dif * vline_salt_dif,
          cross_salt_dif * hline_salt_dif)

# POTENTIAL DENSITY

In [ ]:
# Age averaged through time
ds_dense = dso.pot_rho_0.mean('time')

# Ctrl age averaged through time
ds_dense_c = dso_c.pot_rho_0.mean('time')

# Differences
ds_dense_dif = ds_dense - ds_dense_c#.fillna(0)
#ds_age_dif.compute()

In [ ]:
hv.extension('bokeh')

colorbar_range_dense = (1026, 1029)
cmap_dense = "cmo.dense"
initial_lon = -68.5

# Stream to follow the pointer over the upper plot (and then update the lower plot's cross-section)
posx_dense = hv.streams.PointerX()

# ~~~~~ ~~~~~ ~~~~~ ~~~~~ ~~~~~ ^_^ ~~~~~ ~~~~~ ~~~~~ ~~~~~ ~~~~~
# WIDGETS

# Widget to select the DEPTH
depth_slider_dense = pn.widgets.DiscreteSlider(name='Depth', options=ds_dense.st_ocean.values.tolist(), 
                                             value=ds_dense.st_ocean.values[0], width=500, formatter='%.2f (m)')

# Widget to select the COLORBAR RANGE
colorbar_range_slider_dense = pn.widgets.RangeSlider(name='Colorbar Range', start=colorbar_range_dense[0], 
                                                    end=colorbar_range_dense[1],
                                                    value=colorbar_range_dense, step=0.1, width=500)

# Widget to activate a DISCRETE COLORBAR (instead of a continuous one)
discrete_colorbar_toggle_dense = pn.widgets.Toggle(name='Show Discrete Colors', width=150)

# Widget to activate a HIGH-CONTRAST COLORBAR (instead of a continuous one)
contrast_colorbar_toggle_dense = pn.widgets.Toggle(name='Show High-Contrast Colors', width=150)

# Widget to select the NUMBER of BINS of the colorbar
colorbins_slider_dense = pn.widgets.IntSlider(name='Colorbar # Bins', start=5, end=30, step=1, value=15,
                                            width=200)

# ~~~~~ ~~~~~ ~~~~~ ~~~~~ ~~~~~ ^_^ ~~~~~ ~~~~~ ~~~~~ ~~~~~ ~~~~~
# WIDGETS's LINKS

# Attach a link to the widget value, so it automatically refreshes the vertical, 
# longitude line upon change
@pn.depends(high_contrast=contrast_colorbar_toggle_dense.param.value)
def update_vline_dense(x, high_contrast):
    line = hv.VLine(x or initial_lon)
    line.opts(color="Black" if high_contrast else "Crimson", line_dash='dashed')
    return line

# Attach a link to the widget value, so it automatically refreshes the horizontal, 
# depth line upon change
@pn.depends(depth=depth_slider_dense.param.value,
            high_contrast=contrast_colorbar_toggle_dense.param.value)
def update_hline_dense(depth, high_contrast):
    line = hv.HLine(depth)
    line.opts(color="Black" if high_contrast else "Crimson", line_dash='dashed')
    return line

# Watch for changes on the widgets' values to update the plots
@pn.depends(depth=depth_slider_dense.param.value, 
            col_range=colorbar_range_slider_dense.param.value,
            discrete_colors=discrete_colorbar_toggle_dense.param.value,
            bin_count=colorbins_slider_dense.param.value,
            high_contrast=contrast_colorbar_toggle_dense.param.value)
def update_plot_dense(depth, col_range, discrete_colors, bin_count, high_contrast):
    plot = hv.QuadMesh(ds_dense.loc[depth], kdims=["xt_ocean", "yt_ocean"], 
                       vdims=hv.Dimension('temp', range=col_range))
    if high_contrast:
        plot.opts(cmap=Prism_8.hex_colors if discrete_colors else Prism_8.mpl_colormap)
    else:
        plot.opts(cmap=plt.cm.get_cmap(cmap_dense, bin_count) if discrete_colors else cmap_dense)
    plot.opts(title='Density at %.2f(m) depth'% (depth), framewise=True, colorbar=True, 
              clabel=dso.pot_rho_0.units, width=900, height=600, tools=['hover'], yaxis=None, xlabel='Longitude (°E)', 
              bgcolor='Silver', toolbar="above")
    return plot

# Watch for changes on the widgets' values to update the plots
@pn.depends(depth=depth_slider_dense.param.value, 
            col_range=colorbar_range_slider_dense.param.value,
            discrete_colors=discrete_colorbar_toggle_dense.param.value,
            bin_count=colorbins_slider_dense.param.value,
            high_contrast=contrast_colorbar_toggle_dense.param.value)
def update_cross_dense(x, depth, col_range, discrete_colors, bin_count, high_contrast):
    longitud = closest_lon(x or (initial_lon))
    
    plot = hv.QuadMesh(ds_dense.sel(xt_ocean=longitud), 
                       kdims=["yt_ocean", "st_ocean"], 
                       vdims=hv.Dimension('temp', range=col_range))
    if high_contrast:
        plot.opts(cmap=Prism_8.hex_colors if discrete_colors else Prism_8.mpl_colormap)
    else:
        plot.opts(cmap=plt.cm.get_cmap(cmap_dense, bin_count) if discrete_colors else cmap_dense)
    plot.opts(title='Density cross-section at %.1f°E'% (longitud), framewise=True, invert_yaxis=True, 
              colorbar=True, clabel=dso.pot_rho_0.units, bgcolor='Silver', tools=['hover'], width=900, height=450,
              ylabel='Depth (m)', xlabel='Latitude (°N)', toolbar="above")
    return plot

# ~~~~~ ~~~~~ ~~~~~ ~~~~~ ~~~~~ ^_^ ~~~~~ ~~~~~ ~~~~~ ~~~~~ ~~~~~
# LONGITUDE LINE

vline_dense = hv.DynamicMap(update_vline_dense, streams=[posx_dense])

# DEPTH LINE
hline_dense = hv.DynamicMap(update_hline_dense)

# U ANTARCTIC PLOT
plot_dense = hv.DynamicMap(update_plot_dense)

# U CROSS-SECTION
cross_dense = hv.DynamicMap(update_cross_dense, streams=[posx_dense])

# ~~~~~ ~~~~~ ~~~~~ ~~~~~ ~~~~~ ^_^ ~~~~~ ~~~~~ ~~~~~ ~~~~~ ~~~~~
# Build the layout, using Spacers as padding amongst elements 
pn.Column(pn.Row(pn.Spacer(width=50), depth_slider_dense, discrete_colorbar_toggle_dense, contrast_colorbar_toggle_dense), 
          pn.Row(pn.Spacer(width=50), colorbar_range_slider_dense, colorbins_slider_dense),
          pn.Spacer(height=5),
          plot_dense * vline_dense,
          cross_dense * hline_dense)

# <font color=gray>DiFFERENCES IN POTENTIAL DENSITY</font>

In [ ]:
hv.extension('bokeh')

colorbar_range_dense_dif = (-0.5, 0.5)
cmap_dense_dif = "RdBu_r"
initial_lon = -68.5

# Stream to follow the pointer over the upper plot (and then update the lower plot's cross-section)
posx_dense_dif = hv.streams.PointerX()

# ~~~~~ ~~~~~ ~~~~~ ~~~~~ ~~~~~ ^_^ ~~~~~ ~~~~~ ~~~~~ ~~~~~ ~~~~~
# WIDGETS

# Widget to select the DEPTH
depth_slider_dense_dif = pn.widgets.DiscreteSlider(name='Depth', options=ds_dense_dif.st_ocean.values.tolist(), 
                                             value=ds_dense_dif.st_ocean.values[0], width=500, formatter='%.2f (m)')

# Widget to select the COLORBAR RANGE
colorbar_slider_dense_dif = pn.widgets.FloatSlider(name='Colorbar Range', start=0, end=colorbar_range_dense_dif[1], 
                                                  step=0.01, value=0.25, width=500)

# Widget to activate a DISCRETE COLORBAR (instead of a continuous one)
discrete_colorbar_toggle_dense_dif = pn.widgets.Toggle(name='Show Discrete Colors', width=150)

# Widget to select the NUMBER of BINS of the colorbar
colorbins_slider_dense_dif = pn.widgets.IntSlider(name='Colorbar # Bins', start=5, end=30, step=1, value=15,
                                            width=200)

# ~~~~~ ~~~~~ ~~~~~ ~~~~~ ~~~~~ ^_^ ~~~~~ ~~~~~ ~~~~~ ~~~~~ ~~~~~
# WIDGETS's LINKS

# Attach a link to the widget value, so it automatically refreshes the horizontal, 
# depth line upon change
@pn.depends(depth=depth_slider_dense_dif.param.value)
def update_hline_dense_dif(depth):
    return hv.HLine(depth)

# Watch for changes on the widgets' values to update the plots
@pn.depends(depth=depth_slider_dense_dif.param.value, 
            col_range=colorbar_slider_dense_dif.param.value,
            discrete_colors=discrete_colorbar_toggle_dense_dif.param.value,
            bin_count=colorbins_slider_dense_dif.param.value)
def update_plot_dense_dif(depth, col_range, discrete_colors, bin_count):
    plot = hv.QuadMesh(ds_dense_dif.loc[depth], kdims=["xt_ocean", "yt_ocean"], 
                       vdims=hv.Dimension('rho', range=(-col_range, col_range)))
    plot.opts(cmap=plt.cm.get_cmap(cmap_dense_dif, bin_count) if discrete_colors else cmap_dense_dif)
    plot.opts(title='Differences in Density at %.2f(m) depth'% (depth), framewise=True, colorbar=True, 
              clabel=dso.pot_rho_0.units, width=900, height=600, tools=['hover'], yaxis=None, xlabel='Longitude (°E)', 
              bgcolor='Silver', toolbar="above")
    return plot

# Watch for changes on the widgets' values to update the plots
@pn.depends(depth=depth_slider_dense_dif.param.value, 
            col_range=colorbar_slider_dense_dif.param.value,
            discrete_colors=discrete_colorbar_toggle_dense_dif.param.value,
            bin_count=colorbins_slider_dense_dif.param.value)
def update_cross_dense_dif(x, depth, col_range, discrete_colors, bin_count):
    longitud = closest_lon(x or (initial_lon))
    
    plot = hv.QuadMesh(ds_dense_dif.sel(xt_ocean=longitud), 
                       kdims=["yt_ocean", "st_ocean"], 
                       vdims=hv.Dimension('rho', range=(-col_range, col_range)))
    plot.opts(cmap=plt.cm.get_cmap(cmap_dense_dif, bin_count) if discrete_colors else cmap_dense_dif)
    plot.opts(title='Differences in Density: cross-section at %.1f°E'% (longitud), framewise=True, invert_yaxis=True, 
              colorbar=True, clabel=dso.pot_rho_0.units, bgcolor='Silver', tools=['hover'], width=900, height=450,
              ylabel='Depth (m)', xlabel='Latitude (°N)', toolbar="above")
    return plot

# ~~~~~ ~~~~~ ~~~~~ ~~~~~ ~~~~~ ^_^ ~~~~~ ~~~~~ ~~~~~ ~~~~~ ~~~~~
# LONGITUDE LINE

vline_dense_dif = hv.DynamicMap(lambda x: hv.VLine(x or initial_lon), streams=[posx_dense_dif])
vline_dense_dif.opts(color="MediumSeaGreen", line_dash='dashed')

# DEPTH LINE
hline_dense_dif = hv.DynamicMap(update_hline_dense_dif)
hline_dense_dif.opts(color="MediumSeaGreen", line_dash='dashed')

# U ANTARCTIC PLOT
plot_dense_dif = hv.DynamicMap(update_plot_dense_dif)

# U CROSS-SECTION
cross_dense_dif = hv.DynamicMap(update_cross_dense_dif, streams=[posx_dense_dif])

# ~~~~~ ~~~~~ ~~~~~ ~~~~~ ~~~~~ ^_^ ~~~~~ ~~~~~ ~~~~~ ~~~~~ ~~~~~
# Build the layout, using Spacers as padding amongst elements 
pn.Column(pn.Row(pn.Spacer(width=50), depth_slider_dense_dif, discrete_colorbar_toggle_dense_dif), 
          pn.Row(pn.Spacer(width=50), colorbar_slider_dense_dif, colorbins_slider_dense_dif),
          pn.Spacer(height=5),
          plot_dense_dif * vline_dense_dif,
          cross_dense_dif * hline_dense_dif)

# WATER's AGE

In [ ]:
# Age averaged through time
ds_age = dsm.age_global.mean('time')

# Ctrl age averaged through time
ds_age_c = dsm_c.age_global.mean('time')

# Differences
ds_age_dif = ds_age - ds_age_c
#ds_age_dif.compute()

In [ ]:
hv.extension('bokeh')

colorbar_range_age = (0, 320)
cmap_age = "cmo.solar_r"
initial_lon = -68.5

# Stream to follow the pointer over the upper plot (and then update the lower plot's cross-section)
posx_age = hv.streams.PointerX()

# ~~~~~ ~~~~~ ~~~~~ ~~~~~ ~~~~~ ^_^ ~~~~~ ~~~~~ ~~~~~ ~~~~~ ~~~~~
# WIDGETS

# Widget to select the DEPTH
depth_slider_age = pn.widgets.DiscreteSlider(name='Depth', options=ds_age.st_ocean.values.tolist(), 
                                             value=ds_age.st_ocean.values[0], width=500, formatter='%.2f (m)')

# Widget to select the COLORBAR RANGE
colorbar_range_slider_age = pn.widgets.RangeSlider(name='Colorbar Range', start=colorbar_range_age[0], 
                                                    end=colorbar_range_age[1],
                                                    value=colorbar_range_age, step=0.1, width=500)

# Widget to activate a DISCRETE COLORBAR (instead of a continuous one)
discrete_colorbar_toggle_age = pn.widgets.Toggle(name='Show Discrete Colors', width=150)

# Widget to activate a HIGH-CONTRAST COLORBAR (instead of a continuous one)
contrast_colorbar_toggle_age = pn.widgets.Toggle(name='Show High-Contrast Colors', width=150)

# Widget to select the NUMBER of BINS of the colorbar
colorbins_slider_age = pn.widgets.IntSlider(name='Colorbar # Bins', start=5, end=30, step=1, value=15,
                                            width=200)

# ~~~~~ ~~~~~ ~~~~~ ~~~~~ ~~~~~ ^_^ ~~~~~ ~~~~~ ~~~~~ ~~~~~ ~~~~~
# WIDGETS's LINKS

# Attach a link to the widget value, so it automatically refreshes the vertical, 
# longitude line upon change
@pn.depends(high_contrast=contrast_colorbar_toggle_age.param.value)
def update_vline_age(x, high_contrast):
    line = hv.VLine(x or initial_lon)
    line.opts(color="Black" if high_contrast else "RoyalBlue", line_dash='dashed')
    return line

# Attach a link to the widget value, so it automatically refreshes the horizontal, 
# depth line upon change
@pn.depends(depth=depth_slider_age.param.value,
            high_contrast=contrast_colorbar_toggle_age.param.value)
def update_hline_age(depth, high_contrast):
    line = hv.HLine(depth)
    line.opts(color="Black" if high_contrast else "RoyalBlue", line_dash='dashed')
    return line

# Watch for changes on the widgets' values to update the plots
@pn.depends(depth=depth_slider_age.param.value, 
            col_range=colorbar_range_slider_age.param.value,
            discrete_colors=discrete_colorbar_toggle_age.param.value,
            bin_count=colorbins_slider_age.param.value,
            high_contrast=contrast_colorbar_toggle_age.param.value)
def update_plot_age(depth, col_range, discrete_colors, bin_count, high_contrast):
    plot = hv.QuadMesh(ds_age.loc[depth], kdims=["xt_ocean", "yt_ocean"], 
                       vdims=hv.Dimension('age', range=col_range))
    if high_contrast:
        plot.opts(cmap=Prism_8.hex_colors if discrete_colors else Prism_8.mpl_colormap)
    else:
        plot.opts(cmap=plt.cm.get_cmap(cmap_age, bin_count) if discrete_colors else cmap_age)
    plot.opts(title="Water's age at %.2f(m) depth"% (depth), framewise=True, colorbar=True, 
              clabel="yr", width=900, height=600, tools=['hover'], yaxis=None, xlabel='Longitude (°E)', 
              bgcolor='Silver', toolbar="above")
    return plot

# Watch for changes on the widgets' values to update the plots
@pn.depends(depth=depth_slider_age.param.value, 
            col_range=colorbar_range_slider_age.param.value,
            discrete_colors=discrete_colorbar_toggle_age.param.value,
            bin_count=colorbins_slider_age.param.value,
            high_contrast=contrast_colorbar_toggle_age.param.value)
def update_cross_age(x, depth, col_range, discrete_colors, bin_count, high_contrast):
    longitud = closest_lon(x or (initial_lon))
    
    plot = hv.QuadMesh(ds_age.sel(xt_ocean=longitud), 
                       kdims=["yt_ocean", "st_ocean"], 
                       vdims=hv.Dimension('age', range=col_range))
    if high_contrast:
        plot.opts(cmap=Prism_8.hex_colors if discrete_colors else Prism_8.mpl_colormap)
    else:
        plot.opts(cmap=plt.cm.get_cmap(cmap_age, bin_count) if discrete_colors else cmap_age)
    plot.opts(title="Water's age cross-section at %.1f°E"% (longitud), framewise=True, invert_yaxis=True, 
              colorbar=True, clabel="yr", bgcolor='Silver', tools=['hover'], width=900, height=450,
              ylabel='Depth (m)', xlabel='Latitude (°N)', toolbar="above")
    return plot

# ~~~~~ ~~~~~ ~~~~~ ~~~~~ ~~~~~ ^_^ ~~~~~ ~~~~~ ~~~~~ ~~~~~ ~~~~~
# LONGITUDE LINE

vline_age = hv.DynamicMap(update_vline_age, streams=[posx_age])

# DEPTH LINE
hline_age = hv.DynamicMap(update_hline_age)

# U ANTARCTIC PLOT
plot_age = hv.DynamicMap(update_plot_age)

# U CROSS-SECTION
cross_age = hv.DynamicMap(update_cross_age, streams=[posx_age])

# ~~~~~ ~~~~~ ~~~~~ ~~~~~ ~~~~~ ^_^ ~~~~~ ~~~~~ ~~~~~ ~~~~~ ~~~~~
# Build the layout, using Spacers as padding amongst elements 
pn.Column(pn.Row(pn.Spacer(width=50), depth_slider_age, discrete_colorbar_toggle_age, contrast_colorbar_toggle_age), 
          pn.Row(pn.Spacer(width=50), colorbar_range_slider_age, colorbins_slider_age),
          pn.Spacer(height=5),
          plot_age * vline_age,
          cross_age * hline_age)

# <font color=gray>DiFFERENCES in WATER's AGE</font>

In [ ]:
hv.extension('bokeh')

colorbar_range_age_dif = (-20, 20)
cmap_age_dif = "RdBu_r"
initial_lon = -68.5

# Stream to follow the pointer over the upper plot (and then update the lower plot's cross-section)
posx_age_dif = hv.streams.PointerX()

# ~~~~~ ~~~~~ ~~~~~ ~~~~~ ~~~~~ ^_^ ~~~~~ ~~~~~ ~~~~~ ~~~~~ ~~~~~
# WIDGETS

# Widget to select the DEPTH
depth_slider_age_dif = pn.widgets.DiscreteSlider(name='Depth', options=ds_age_dif.st_ocean.values.tolist(), 
                                             value=ds_age_dif.st_ocean.values[0], width=500, formatter='%.2f (m)')

# Widget to select the COLORBAR RANGE
colorbar_slider_age_dif = pn.widgets.FloatSlider(name='Colorbar Range', start=0, end=colorbar_range_age_dif[1], 
                                                  step=1, value=18, width=500)

# Widget to activate a DISCRETE COLORBAR (instead of a continuous one)
discrete_colorbar_toggle_age_dif = pn.widgets.Toggle(name='Show Discrete Colors', width=150)

# Widget to select the NUMBER of BINS of the colorbar
colorbins_slider_age_dif = pn.widgets.IntSlider(name='Colorbar # Bins', start=5, end=30, step=1, value=15,
                                            width=200)

# ~~~~~ ~~~~~ ~~~~~ ~~~~~ ~~~~~ ^_^ ~~~~~ ~~~~~ ~~~~~ ~~~~~ ~~~~~
# WIDGETS's LINKS

# Attach a link to the widget value, so it automatically refreshes the horizontal, 
# depth line upon change
@pn.depends(depth=depth_slider_age_dif.param.value)
def update_hline_age_dif(depth):
    return hv.HLine(depth)

# Watch for changes on the widgets' values to update the plots
@pn.depends(depth=depth_slider_age_dif.param.value, 
            col_range=colorbar_slider_age_dif.param.value,
            discrete_colors=discrete_colorbar_toggle_age_dif.param.value,
            bin_count=colorbins_slider_age_dif.param.value)
def update_plot_age_dif(depth, col_range, discrete_colors, bin_count):
    plot = hv.QuadMesh(ds_age_dif.loc[depth], kdims=["xt_ocean", "yt_ocean"], 
                       vdims=hv.Dimension('age_global', range=(-col_range, col_range)))
    plot.opts(cmap=plt.cm.get_cmap(cmap_age_dif, bin_count) if discrete_colors else cmap_age_dif)
    plot.opts(title="Differences in Water's Age at %.2f(m) depth"% (depth), framewise=True, colorbar=True, 
              clabel="yr", width=900, height=600, tools=['hover'], yaxis=None, xlabel='Longitude (°E)', 
              bgcolor='Silver', toolbar="above")
    return plot

# Watch for changes on the widgets' values to update the plots
@pn.depends(depth=depth_slider_age_dif.param.value, 
            col_range=colorbar_slider_age_dif.param.value,
            discrete_colors=discrete_colorbar_toggle_age_dif.param.value,
            bin_count=colorbins_slider_age_dif.param.value)
def update_cross_age_dif(x, depth, col_range, discrete_colors, bin_count):
    longitud = closest_lon(x or (initial_lon))
    
    plot = hv.QuadMesh(ds_age_dif.sel(xt_ocean=longitud), 
                       kdims=["yt_ocean", "st_ocean"], 
                       vdims=hv.Dimension('age_global', range=(-col_range, col_range)))
    plot.opts(cmap=plt.cm.get_cmap(cmap_age_dif, bin_count) if discrete_colors else cmap_age_dif)
    plot.opts(title="Differences in Water's Age: cross-section at %.1f°E"% (longitud), framewise=True, invert_yaxis=True, 
              colorbar=True, clabel="yr", bgcolor='Silver', tools=['hover'], width=900, height=450,
              ylabel='Depth (m)', xlabel='Latitude (°N)', toolbar="above")
    return plot

# ~~~~~ ~~~~~ ~~~~~ ~~~~~ ~~~~~ ^_^ ~~~~~ ~~~~~ ~~~~~ ~~~~~ ~~~~~
# LONGITUDE LINE

vline_age_dif = hv.DynamicMap(lambda x: hv.VLine(x or initial_lon), streams=[posx_age_dif])
vline_age_dif.opts(color="MediumSeaGreen", line_dash='dashed')

# DEPTH LINE
hline_age_dif = hv.DynamicMap(update_hline_age_dif)
hline_age_dif.opts(color="MediumSeaGreen", line_dash='dashed')

# U ANTARCTIC PLOT
plot_age_dif = hv.DynamicMap(update_plot_age_dif)

# U CROSS-SECTION
cross_age_dif = hv.DynamicMap(update_cross_age_dif, streams=[posx_age_dif])

# ~~~~~ ~~~~~ ~~~~~ ~~~~~ ~~~~~ ^_^ ~~~~~ ~~~~~ ~~~~~ ~~~~~ ~~~~~
# Build the layout, using Spacers as padding amongst elements 
pn.Column(pn.Row(pn.Spacer(width=50), depth_slider_age_dif, discrete_colorbar_toggle_age_dif), 
          pn.Row(pn.Spacer(width=50), colorbar_slider_age_dif, colorbins_slider_age_dif),
          pn.Spacer(height=5),
          plot_age_dif * vline_age_dif,
          cross_age_dif * hline_age_dif)

# WATER's AGE through TIME

In [ ]:
# Age averaged through time
ds_age_global = dsm.age_global#.mean('time')

# Ctrl age averaged through time
ds_age_global_c = dsm_c.age_global#.mean('time')

# Differences
ds_age_global_dif = ds_age_global - ds_age_global_c
#ds_age_global_dif.compute()

In [ ]:
# Transform the USELESS! cftime dates into simple strings so we can actually work with them... ¬_¬
dt = ds_age_global_dif.time.values.astype('datetime64[s]')
dates = np.asarray([str(e).split('T')[0][:7] for e in dt]) # Select just year-month

In [ ]:
import cftime
hv.extension('bokeh')
colorbar_range_age_dif = (-30, 30)
cmap_age_dif = "RdBu_r"
initial_lon = -68.5

# Stream to follow the pointer over the upper plot (and then update the lower plot's cross-section)
posx_age_dif = hv.streams.PointerX()

# ~~~~~ ~~~~~ ~~~~~ ~~~~~ ~~~~~ ^_^ ~~~~~ ~~~~~ ~~~~~ ~~~~~ ~~~~~
# WIDGETS

# Widget to select TIME
time_slider_age_dif = pn.widgets.DiscreteSlider(name='Time', options=dates.tolist(),
                                            value=dates[5].tolist(), width=500)

# Widget to select the DEPTH
depth_slider_age_dif = pn.widgets.DiscreteSlider(name='Depth', options=ds_age_global_dif.st_ocean.values.tolist(), 
                                             value=ds_age_global_dif.st_ocean.values[-16].tolist(), width=500, formatter='%.2f (m)')

# Widget to select the COLORBAR RANGE
colorbar_slider_age_dif = pn.widgets.FloatSlider(name='Colorbar Range', start=0, end=colorbar_range_age_dif[1], 
                                                  step=1, value=18, width=500)

# Widget to activate a DISCRETE COLORBAR (instead of a continuous one)
discrete_colorbar_toggle_age_dif = pn.widgets.Toggle(name='Show Discrete Colors', width=150)

# Widget to select the NUMBER of BINS of the colorbar
colorbins_slider_age_dif = pn.widgets.IntSlider(name='Colorbar # Bins', start=5, end=30, step=1, value=15,
                                            width=200)

# ~~~~~ ~~~~~ ~~~~~ ~~~~~ ~~~~~ ^_^ ~~~~~ ~~~~~ ~~~~~ ~~~~~ ~~~~~
# WIDGETS's LINKS

# Attach a link to the widget value, so it automatically refreshes the horizontal, 
# depth line upon change
@pn.depends(depth=depth_slider_age_dif.param.value)
def update_hline_age_dif(depth):
    return hv.HLine(depth)

# Watch for changes on the widgets' values to update the plots
@pn.depends(tim=time_slider_age_dif.param.value,
            depth=depth_slider_age_dif.param.value, 
            col_range=colorbar_slider_age_dif.param.value,
            discrete_colors=discrete_colorbar_toggle_age_dif.param.value,
            bin_count=colorbins_slider_age_dif.param.value)
def update_plot_age_dif(tim, depth, col_range, discrete_colors, bin_count):
    plot = hv.QuadMesh(ds_age_global_dif.sel(time=tim, st_ocean=depth), kdims=["xt_ocean", "yt_ocean"], 
                       vdims=hv.Dimension('age_global', range=(-col_range, col_range)))
    plot.opts(cmap=plt.cm.get_cmap(cmap_age_dif, bin_count) if discrete_colors else cmap_age_dif)
    plot.opts(title="Differences in Water's Age at %.2f(m) depth during %s"% (depth,tim), framewise=True, colorbar=True, 
              clabel="yr", width=900, height=600, tools=['hover'], yaxis=None, xlabel='Longitude (°E)', 
              bgcolor='Silver', toolbar="above")
    return plot

# Watch for changes on the widgets' values to update the plots
@pn.depends(tim=time_slider_age_dif.param.value,
            depth=depth_slider_age_dif.param.value, 
            col_range=colorbar_slider_age_dif.param.value,
            discrete_colors=discrete_colorbar_toggle_age_dif.param.value,
            bin_count=colorbins_slider_age_dif.param.value)
def update_cross_age_dif(x, tim, depth, col_range, discrete_colors, bin_count):
    longitud = closest_lon(x or (initial_lon))
    
    plot = hv.QuadMesh(ds_age_global_dif.sel(time=tim, xt_ocean=longitud), 
                       kdims=["yt_ocean", "st_ocean"], 
                       vdims=hv.Dimension('age_global', range=(-col_range, col_range)))
    plot.opts(cmap=plt.cm.get_cmap(cmap_age_dif, bin_count) if discrete_colors else cmap_age_dif)
    plot.opts(title="Differences in Water's Age: cross-section at %.1f°E during %s"% (longitud,tim), framewise=True, invert_yaxis=True, 
              colorbar=True, clabel="yr", bgcolor='Silver', tools=['hover'], width=900, height=450,
              ylabel='Depth (m)', xlabel='Latitude (°N)', toolbar="above")
    return plot

# ~~~~~ ~~~~~ ~~~~~ ~~~~~ ~~~~~ ^_^ ~~~~~ ~~~~~ ~~~~~ ~~~~~ ~~~~~
# LONGITUDE LINE

vline_age_dif = hv.DynamicMap(lambda x: hv.VLine(x or initial_lon), streams=[posx_age_dif])
vline_age_dif.opts(color="MediumSeaGreen", line_dash='dashed')

# DEPTH LINE
hline_age_dif = hv.DynamicMap(update_hline_age_dif)
hline_age_dif.opts(color="MediumSeaGreen", line_dash='dashed')

# U ANTARCTIC PLOT
plot_age_dif = hv.DynamicMap(update_plot_age_dif)

# U CROSS-SECTION
cross_age_dif = hv.DynamicMap(update_cross_age_dif, streams=[posx_age_dif])

# ~~~~~ ~~~~~ ~~~~~ ~~~~~ ~~~~~ ^_^ ~~~~~ ~~~~~ ~~~~~ ~~~~~ ~~~~~
# Build the layout, using Spacers as padding amongst elements 
pn.Column(pn.Row(pn.Spacer(width=50), time_slider_age_dif),
          pn.Row(pn.Spacer(width=50), depth_slider_age_dif, discrete_colorbar_toggle_age_dif), 
          pn.Row(pn.Spacer(width=50), colorbar_slider_age_dif, colorbins_slider_age_dif),
          pn.Spacer(height=5),
          plot_age_dif * vline_age_dif,
          cross_age_dif * hline_age_dif)

# ICE VELOCITIES

In [ ]:
S45_index = np.where(ds_ice.TLAT.values < -45)[0][-1]

### YEARLY AVERAGE

In [ ]:
ice_u = ds_ice.uvel_m.mean('time')
ice_v = ds_ice.vvel_m.mean('time')

ice_u_c = ds_ice_c.uvel_m.mean('time')
ice_v_c = ds_ice_c.vvel_m.mean('time')

ice_u_dif = ice_u - ice_u_c.fillna(0)
ice_v_dif = ice_v - ice_v_c.fillna(0)

In [ ]:
# Bipolar Colormap for the arrows
vector_map = colors.ListedColormap(['RoyalBlue', 'Crimson'])
sample_step = 4

plt.figure(figsize=(15, 45))


# CTRL
ax_c = plt.subplot(3, 1, 1, projection=ccrs.Mercator(central_longitude=-80))
ax_c.add_feature(cartopy.feature.OCEAN, zorder=0, color="Azure")
ax_c.add_feature(cartopy.feature.LAND, zorder=0, edgecolor='black', color="Khaki")

ice_magnitudes_c = np.sign(ice_u_c[::sample_step, ::sample_step])

ax_c.quiver(ice_u_c.ULON[::sample_step, ::sample_step].values, ice_u_c.ULAT[::sample_step, ::sample_step].values, 
            ice_u_c[::sample_step, ::sample_step].values, ice_v_c[::sample_step, ::sample_step].values, 
            ice_magnitudes_c.values,  scale=5, cmap=vector_map, transform=ccrs.PlateCarree())

plt.title("Control's ice velocities", fontsize=30, pad=20)


# EXPERIMENT
ax = plt.subplot(3, 1, 2, projection=ccrs.Mercator(central_longitude=-80))
ax.add_feature(cartopy.feature.OCEAN, zorder=0, color="Azure")
ax.add_feature(cartopy.feature.LAND, zorder=0, edgecolor='black', color="Khaki")

ice_magnitudes = np.sign(ice_u[::sample_step, ::sample_step])

ax.quiver(ds_ice.ULON[::sample_step, ::sample_step].values, ds_ice.ULAT[::sample_step, ::sample_step].values, ice_u[::sample_step, ::sample_step].values, ice_v[::sample_step, ::sample_step].values, 
          ice_magnitudes.values,  scale=5, cmap=vector_map, transform=ccrs.PlateCarree())

plt.title("Experiment's ice velocities", fontsize=30, pad=20)


# DIFF
ax_dif = plt.subplot(3, 1, 3, projection=ccrs.Mercator(central_longitude=-80))
ax_dif.add_feature(cartopy.feature.OCEAN, zorder=0, color="Azure")
ax_dif.add_feature(cartopy.feature.LAND, zorder=0, edgecolor='black', color="Khaki")

ice_magnitudes_dif = np.sign(ice_u_dif[::sample_step, ::sample_step])

ax_dif.quiver(ice_u_dif.ULON[::sample_step, ::sample_step].values, ice_u_dif.ULAT[::sample_step, ::sample_step].values, 
              ice_u_dif[::sample_step, ::sample_step].values, ice_v_dif[::sample_step, ::sample_step].values, 
              ice_magnitudes_dif.values,  scale=1.5, cmap=vector_map, transform=ccrs.PlateCarree())

plt.title("Differences in ice velocities", fontsize=30, pad=20)


plt.show()

### ANTARCTICA'S WINTER

In [ ]:
S45_index = np.where(ds_ice.uvel_m.TLAT.values < -45)[0][-1]
ds_ice.uvel_m[:, :S45_index]

In [ ]:
# Bipolar Colormap for the arrows
vector_map = colors.ListedColormap(['RoyalBlue', 'Crimson'])
sample_step = 4

plt.figure(figsize=(15, 15))


# CTRL
# Average per month of the year, then select the winter period, and mean over the whole winter
ice_u_c = ds_ice_c.uvel_m[:, :S45_index].groupby('time.month').mean('time')[8:11].mean('month')
ice_v_c = ds_ice_c.vvel_m[:, :S45_index].groupby('time.month').mean('time')[8:11].mean('month')

ax_c = plt.subplot(3, 1, 1, projection=ccrs.Mercator(central_longitude=-80))
plt.title("Control's ice velocity around Antarctica's Winter", fontsize=30, pad=20)

ax_c.add_feature(cartopy.feature.OCEAN, zorder=0, color='Azure')
ax_c.add_feature(cartopy.feature.LAND, zorder=0, color='Khaki')

ice_magnitudes_c = np.sign(ice_u_c[::sample_step, ::sample_step])
ax_c.quiver(ice_u_c.ULON[::sample_step, ::sample_step].values, ice_u_c.ULAT[::sample_step, ::sample_step].values, 
           ice_u_c[::sample_step, ::sample_step].values, ice_v_c[::sample_step, ::sample_step].values, 
           ice_magnitudes_c.values,  scale=6, cmap=vector_map, 
           transform=ccrs.PlateCarree())


# EXPERIMENT
# Average per month of the year, then select the winter period, and mean over the whole winter
ice_u = ds_ice.uvel_m[:, :S45_index].groupby('time.month').mean('time')[8:11].mean('month')
ice_v = ds_ice.vvel_m[:, :S45_index].groupby('time.month').mean('time')[8:11].mean('month')

ax = plt.subplot(3, 1, 2, projection=ccrs.Mercator(central_longitude=-80))
plt.title("Experiment's ice velocity around Antarctica's Winter", fontsize=30, pad=20)

ax.add_feature(cartopy.feature.OCEAN, zorder=0, color='Azure')
ax.add_feature(cartopy.feature.LAND, zorder=0, color='Khaki')

ice_magnitudes = np.sign(ice_u[::sample_step, ::sample_step])
ax.quiver(ice_u.ULON[::sample_step, ::sample_step].values, ice_u.ULAT[::sample_step, ::sample_step].values, 
           ice_u[::sample_step, ::sample_step].values, ice_v[::sample_step, ::sample_step].values, 
           ice_magnitudes.values,  scale=6, cmap=vector_map, 
           transform=ccrs.PlateCarree())


# DIFF
# Average per month of the year, then select the winter period, and mean over the whole winter
ice_u_dif = ice_u - ice_u_c.fillna(0)
ice_v_dif = ice_v - ice_v_c.fillna(0)

ax_dif = plt.subplot(3, 1, 3, projection=ccrs.Mercator(central_longitude=-80))
plt.title("Differences in ice velocity around Antarctica's Winter", fontsize=30, pad=20)

ax_dif.add_feature(cartopy.feature.OCEAN, zorder=0, color='Azure')
ax_dif.add_feature(cartopy.feature.LAND, zorder=0, color='Khaki')

ice_magnitudes_dif = np.sign(ice_u_dif[::sample_step, ::sample_step])
ax_dif.quiver(ice_u_dif.ULON[::sample_step, ::sample_step].values, ice_u_dif.ULAT[::sample_step, ::sample_step].values, 
           ice_u_dif[::sample_step, ::sample_step].values, ice_v_dif[::sample_step, ::sample_step].values, 
           ice_magnitudes_dif.values,  scale=1.5, cmap=vector_map, 
           transform=ccrs.PlateCarree())


plt.show()

# ICE COVER

In [ ]:
plt.figure(figsize=(15, 12))


# CTRL
ice_cover_c = ds_ice_c.aice_m.sel(yt_ocean=slice(-78,-45)).mean('time')

ax_c = plt.subplot(3, 1, 1, projection=ccrs.Mercator(central_longitude=-80))
# ax_c.add_feature(cartopy.feature.OCEAN, zorder=0, color="Azure")
# ax_c.add_feature(cartopy.feature.LAND, zorder=0, edgecolor='black', color="Khaki")

plot_c = ax_c.contourf(ice_cover_c, 20, cmap=cmo.ice, projection=ccrs.PlateCarree())

plt.title("Control's ice cover (%)", fontsize=30, pad=20)
plt.colorbar(plot_c, ax=ax_c, shrink=0.6)


# EXPERIMENT
ice_cover = ds_ice.aice_m.sel(yt_ocean=slice(-78,-45)).mean('time')

ax = plt.subplot(3, 1, 2, projection=ccrs.Mercator(central_longitude=-80))
# ax.add_feature(cartopy.feature.OCEAN, zorder=0, color="Azure")
# ax.add_feature(cartopy.feature.LAND, zorder=0, edgecolor='black', color="Khaki")

plot = ax.contourf(ice_cover, 20, cmap=cmo.ice, projection=ccrs.PlateCarree())

plt.title("Experiment's ice cover (%)", fontsize=30, pad=20)
plt.colorbar(plot, ax=ax, shrink=0.6)


# DIFF
ax_dif = plt.subplot(3, 1, 3, projection=ccrs.Mercator(central_longitude=-80))
# ax_dif.add_feature(cartopy.feature.OCEAN, zorder=0, color="Azure")
#ax_dif.add_feature(cartopy.feature.LAND, zorder=0, edgecolor='black', color="Black")

plot_dif = ax_dif.contourf(ice_cover - ice_cover_c, 20, cmap="RdBu_r", projection=ccrs.PlateCarree(),
                           vmin= -0.2, vmax=0.2)
ax_dif.contour(ice_cover - ice_cover_c, 20, projection=ccrs.PlateCarree(),
               levels=[0.0], colors='Black', linewidths=0.5)
# ax_dif.clabel(plot_dif, inline = 1, fmt = '%1.1f')
plt.title("Differences in ice cover (%)", fontsize=30, pad=20)
plt.colorbar(plot_dif, ax=ax_dif, shrink=0.6)


plt.show()

# ICE MELT

In [ ]:
plt.figure(figsize=(20, 12))


# CTRL
ice_growth_c = (ds_ice_c.meltb_m.sel(yt_ocean=slice(-78,-45)).mean('time')
              + ds_ice_c.meltl_m.sel(yt_ocean=slice(-78,-45)).mean('time'))

ax_c = plt.subplot(3, 1, 1, projection=ccrs.Mercator(central_longitude=-80))
# ax_c.add_feature(cartopy.feature.OCEAN, zorder=0, color="Azure")
ax_c.add_feature(cartopy.feature.LAND, zorder=0, edgecolor='black', color="Khaki")

plot_c = ax_c.contourf(ice_growth_c, 20, cmap=cmo.ice, projection=ccrs.PlateCarree())
# plot_c = ax_c.contourf(ice_growth_c.xt_ocean, ice_growth_c.yt_ocean, ice_growth_c, 20, cmap=cmo.ice, projection=ccrs.PlateCarree())
# plt.xticks(ds_ice.TLON.xt_ocean.values[::20])

plt.title("Control's ice melt (cm/day)", fontsize=30, pad=20)
plt.colorbar(plot_c, ax=ax_c, shrink=0.6)


# EXPERIMENT
ice_growth = (ds_ice.meltb_m.sel(yt_ocean=slice(-78,-45)).mean('time')
            + ds_ice.meltl_m.sel(yt_ocean=slice(-78,-45)).mean('time'))

ax = plt.subplot(3, 1, 2, projection=ccrs.Mercator(central_longitude=-80))
# ax.add_feature(cartopy.feature.OCEAN, zorder=0, color="Azure")
ax.add_feature(cartopy.feature.LAND, zorder=0, edgecolor='black', color="Khaki")

plot = ax.contourf(ice_growth, 20, cmap=cmo.ice, projection=ccrs.PlateCarree())
# plot = ax.contourf(ice_growth.xt_ocean, ice_growth.yt_ocean, ice_growth, 20, cmap=cmo.ice, projection=ccrs.PlateCarree())
# plt.xticks(ds_ice.TLON.xt_ocean.values[::20])

plt.title("Experiment's ice melt (cm/day)", fontsize=30, pad=20)
plt.colorbar(plot, ax=ax, shrink=0.6)


# DIFF
ax_dif = plt.subplot(3, 1, 3, projection=ccrs.Mercator(central_longitude=-80))
# ax_dif.add_feature(cartopy.feature.OCEAN, zorder=0, color="Azure")
ax_dif.add_feature(cartopy.feature.LAND, zorder=0, edgecolor='black', color="black")

plot_dif = ax_dif.contourf(ice_growth - ice_growth_c.fillna(0), 20, cmap="RdBu_r", projection=ccrs.PlateCarree(),
                           vmin= -0.6, vmax=0.6)
ax_dif.contour(ice_growth - ice_growth_c.fillna(0), 15, projection=ccrs.PlateCarree(),
                levels=[0.0,], colors='Black', linewidths=0.5)
plt.title("Differences in ice melt (cm/day) [Experiment - Ctrl]", fontsize=30, pad=20)
plt.colorbar(plot_dif, ax=ax_dif, shrink=0.6)


plt.show()

# ICE FORMATION

In [ ]:
plt.figure(figsize=(20, 12))


# CTRL
ice_growth_c = (ds_ice_c.frazil_m.sel(yt_ocean=slice(-78,-45)).mean('time')
              + ds_ice_c.congel_m.sel(yt_ocean=slice(-78,-45)).mean('time'))

ax_c = plt.subplot(3, 1, 1, projection=ccrs.Mercator(central_longitude=-80))
# ax_c.add_feature(cartopy.feature.OCEAN, zorder=0, color="Azure")
ax_c.add_feature(cartopy.feature.LAND, zorder=0, edgecolor='black', color="Khaki")

plot_c = ax_c.contourf(ice_growth_c, 20, cmap=cmo.ice, projection=ccrs.PlateCarree())

plt.title("Control's ice growth (cm/day)", fontsize=30, pad=20)
plt.colorbar(plot_c, ax=ax_c, shrink=0.6)


# EXPERIMENT
ice_growth = (ds_ice.frazil_m.sel(yt_ocean=slice(-78,-45)).mean('time')
            + ds_ice.congel_m.sel(yt_ocean=slice(-78,-45)).mean('time'))

ax = plt.subplot(3, 1, 2, projection=ccrs.Mercator(central_longitude=-80))
# ax.add_feature(cartopy.feature.OCEAN, zorder=0, color="Azure")
ax.add_feature(cartopy.feature.LAND, zorder=0, edgecolor='black', color="Khaki")

plot = ax.contourf(ice_growth, 20, cmap=cmo.ice, projection=ccrs.PlateCarree())

plt.title("Experiment's ice growth (cm/day)", fontsize=30, pad=20)
plt.colorbar(plot, ax=ax, shrink=0.6)


# DIFF
ax_dif = plt.subplot(3, 1, 3, projection=ccrs.Mercator(central_longitude=-80))
# ax_dif.add_feature(cartopy.feature.OCEAN, zorder=0, color="Azure")
ax_dif.add_feature(cartopy.feature.LAND, zorder=0, edgecolor='black', color="black")

plot_dif = ax_dif.contourf(ice_growth - ice_growth_c.fillna(0), 15, cmap="RdBu_r", projection=ccrs.PlateCarree(),
                           vmin= -2, vmax=2)
ax_dif.contour(ice_growth - ice_growth_c.fillna(0), 15, projection=ccrs.PlateCarree(),
                levels=[0.0,], colors='Black', linewidths=0.5)
plt.title("Differences in ice growth (cm/day)", fontsize=30, pad=20)
plt.colorbar(plot_dif, ax=ax_dif, shrink=0.6)


plt.show()

TOTAL ICE-GROWTH IN THE REGION (CM/DAY)

In [ ]:
ice_growth = (ds_ice.frazil_m.sel(yt_ocean=slice(-78,-60)).mean('time')
              + ds_ice.congel_m.sel(yt_ocean=slice(-78,-60)).mean('time'))

ice_growth_c = (ds_ice_c.frazil_m.sel(yt_ocean=slice(-78,-60)).mean('time')
              + ds_ice_c.congel_m.sel(yt_ocean=slice(-78,-60)).mean('time'))

ice_cover = ds_ice.aice_m.sel(yt_ocean=slice(-78,-60)).mean('time')
ice_cover_c = ds_ice_c.aice_m.sel(yt_ocean=slice(-78,-60)).mean('time')

In [ ]:
ice_growth_dif = ice_growth - ice_growth_c
ice_growth_dif.name = 'growth' # The values of the xArray must have a "name" so Holoviews can use it

In [ ]:
hv.extension('bokeh')

hv_ds = hv.Dataset(ice_growth_dif)

qm = hv_ds.to(hv.QuadMesh, kdims=["xt_ocean", "yt_ocean"], dynamic=True)
qm = qm.redim.range(growth=(-2, 2))

qm.opts(title = "Differences in ice growth (cm/day)", cmap="RdBu_r", width=900, height=300, colorbar=True, 
        tools=['hover'])

qm